In [2]:
import json
import csv
import pandas as pd

In [3]:
path='/Users/fan/Desktop/Fullhouse/WCM/HospitalManagement/OrderAnalysis-master/'

### Load datasets

In [195]:
comorbidityscore = pd.read_excel(path + '2_ComorbidityScore_withduplicates.xlsx', index=None).iloc[:,1:]

In [196]:
comorbidityscore

,ID,Elixhauser,Charlson
0,556943700270,1,1
1,561002000270,1,0
2,562000000270,3,2
3,566520000270,2,1
4,567313100270,5,1
...,...,...,...
2286,1284013900270,3,3
2287,1284311200270,2,1
2288,1284903800270,3,1
2289,1284909400270,2,1


In [197]:
clientvisit = pd.read_csv(path + '2_visit_EDDC_2012_2018.csv', header = 0)
clientvisit.head()

,DischargeDCDesc,DischargeDtm,AdmitDtm,ClientVisitGUID,ClientGUID,HospitalService,EDSite,SummaryDispoCategory,FinalDispoCategory
0,ROUTINE/HOME,2015-11-13 19:03:00.000,2015-11-01 15:20:00.000,805358900270,199802700200,MED,LMH ED,Admitted,Admitted
1,DISCHARGE HOME WITH HM HLTH SERV,2014-02-28 17:55:00.000,2014-02-06 21:06:00.000,579333200270,151733300200,MED,Cornell ED,Admitted,Admitted
2,ROUTINE/HOME,2014-09-12 15:25:00.000,2014-09-10 18:31:00.000,654172500270,3370300200,MED,LMH ED,Admitted,Admitted
3,PATIENT EXPIRED,2015-01-20 11:50:00.000,2015-01-13 18:27:00.000,697843400270,181146900200,MED,LMH ED,Admitted,Admitted
4,ROUTINE/HOME,2017-12-04 13:41:00.000,2017-12-01 19:47:00.000,1092289600270,231826600200,MED,LMH ED,Admitted,Admitted


In [198]:
demo = pd.read_csv(path + '2_demographics_EDDC_2012_2018.csv', header = 0)
demo.head()

,clientguid,clientvisitguid,ethnicity,Ethnicity2,BirthYearNum,LanguageCode,GenderCode,RaceCode,MaritalStatusCode
0,1014700200,721880800270,N,N,1947,English,Female,White,Single
1,1052300200,731006600270,N,NaN,1925,English,Female,Other Not Described,Widower
2,1053600200,1199818100270,N,NaN,1961,English,Male,Black/African Am.,Single
3,1060100200,1092703500270,U,NaN,1918,English,Male,Declined,Married
4,1086000200,974781900270,N,N,1934,English,Female,White,Single


In [3]:
#cluster = pd.read_csv(path + 'cluster_k=3.csv', header = 0)
#insurance2 = pd.read_csv(path + 'Data/insurance_2012_2018_pull2_new2.csv', header = 0)

In [199]:
comorbidityscore.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2291 entries, 0 to 2290
Data columns (total 3 columns):
ID            2291 non-null int64
Elixhauser    2291 non-null int64
Charlson      2291 non-null int64
dtypes: int64(3)
memory usage: 53.8 KB


In [200]:
clientvisit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14044 entries, 0 to 14043
Data columns (total 9 columns):
DischargeDCDesc         13640 non-null object
DischargeDtm            13580 non-null object
AdmitDtm                14044 non-null object
ClientVisitGUID         14044 non-null int64
ClientGUID              14044 non-null int64
HospitalService         13454 non-null object
EDSite                  11136 non-null object
SummaryDispoCategory    11136 non-null object
FinalDispoCategory      11136 non-null object
dtypes: int64(2), object(7)
memory usage: 987.6+ KB


In [8]:
#cluster.info()

In [9]:
#cluster.columns = ['clientvisitguid', 'cluster_by30']
#cluster.info()

In [201]:
demo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13437 entries, 0 to 13436
Data columns (total 9 columns):
clientguid           13437 non-null int64
clientvisitguid      13437 non-null int64
ethnicity            13433 non-null object
Ethnicity2           6612 non-null object
BirthYearNum         13437 non-null int64
LanguageCode         13436 non-null object
GenderCode           13437 non-null object
RaceCode             13437 non-null object
MaritalStatusCode    13436 non-null object
dtypes: int64(3), object(6)
memory usage: 944.9+ KB


In [9]:
#insurance2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15999 entries, 0 to 15998
Data columns (total 2 columns):
clientvisitguid    15999 non-null int64
Financial Group    15999 non-null object
dtypes: int64(1), object(1)
memory usage: 250.1+ KB


#### Load userQ, VTE, CCW 

In [4]:
userq = pd.read_excel(path + '2_Descriptive_stats_0305_cv3_notcancelled.xlsx', sheet_name = 'userQ_cv3_notcancelled', header=None, names = ['clientvisitguid'])
userq = userq.clientvisitguid.str.split(expand = True)

In [5]:
userq.columns = ['clientvisitguid', 'userQ']
userq['clientvisitguid'] = userq['clientvisitguid'].astype(int)
userq.head()

,clientvisitguid,userQ
0,582002300270,Q1
1,685526500270,Q1
2,763976700270,Q1
3,774599200270,Q1
4,789394000270,Q1


In [6]:
userq.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 650 entries, 0 to 649
Data columns (total 2 columns):
clientvisitguid    650 non-null int64
userQ              650 non-null object
dtypes: int64(1), object(1)
memory usage: 10.3+ KB


In [7]:
#userq.to_excel(path + 'userQ_cv3_notcancelled.xlsx', index=None)

In [10]:
#ospercent_userq = pd.read_excel(path + '2_OSpercent_userQ_cv3_notcancelled.xlsx')

In [11]:
#ospercent_userq

,ylclientvisitguid,clientvisitguid,OSperent_EDDC
0,1000120800270,9001000120800270,0.581081
1,1000124200270,9001000124200270,0.466165
2,1000155300270,9001000155300270,0.577381
3,1000634700270,9001000634700270,0.492063
4,1001023900270,9001001023900270,0.542714
...,...,...,...
1646,999130900270,9000999130900270,0.462810
1647,999139900270,9000999139900270,0.458716
1648,999619900270,9000999619900270,0.479638
1649,999912100270,9000999912100270,0.557252


In [12]:
#ospercent_userq1 = pd.merge(ospercent_userq, userq, left_on = 'ylclientvisitguid', right_on = 'clientvisitguid', how='left')

In [14]:
#ospercent_userq1.to_excel(path + '2_OSpercent_userQ_cv3_notcancelled1.xlsx', index=None)

In [205]:
ccw = pd.read_excel(path + '2_Descriptive_stats.xlsx', sheet_name = 'CCW', header=None, names = ['clientvisitguid'])
ccw = ccw.clientvisitguid.str.split(expand = True)

In [206]:
ccw.columns = ['clientvisitguid', 'CCW']
ccw['clientvisitguid'] = ccw['clientvisitguid'].astype(int)
ccw.head()

,clientvisitguid,CCW
0,556943700270,1
1,561002000270,0
2,562000000270,0
3,567313100270,3
4,579333200270,2


In [207]:
ccw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1704 entries, 0 to 1703
Data columns (total 2 columns):
clientvisitguid    1704 non-null int64
CCW                1704 non-null object
dtypes: int64(1), object(1)
memory usage: 26.8+ KB


In [208]:
vte = pd.read_excel(path + '2_Descriptive_stats.xlsx', sheet_name = 'VTE', header=None, names = ['clientvisitguid'])
vte = vte.clientvisitguid.str.split(expand = True)

In [209]:
vte.columns = ['clientvisitguid', 'VTE']
vte['clientvisitguid'] = vte['clientvisitguid'].astype(int)
vte.head()

,clientvisitguid,VTE
0,556943700270,0
1,561002000270,0
2,562000000270,0
3,567313100270,1
4,579333200270,0


The method to get age:
(1) for pid in data2:    ## 2_EDDC_HF_data_filtered_OS.json
        for date in data2[pid]['appt']:
              age = int(date[0:4]) - data2[pid]['age']
              print(pid, age)
(2) For each patient, get the maximun of his/her ages

In [210]:
age = pd.read_excel(path + '2_Descriptive_stats.xlsx', sheet_name = 'age', header=None, names = ['clientvisitguid'])
age = age.clientvisitguid.str.split(expand = True)

In [211]:
age.columns = ['clientvisitguid', 'age']
age['clientvisitguid'] = age['clientvisitguid'].astype(int)
age

,clientvisitguid,age
0,556943700270,84
1,561002000270,85
2,562000000270,81
3,567313100270,88
4,579333200270,64
...,...,...
1699,1283883400270,69
1700,1284311200270,89
1701,1284903800270,82
1702,1284909400270,84


In [212]:
age.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1704 entries, 0 to 1703
Data columns (total 2 columns):
clientvisitguid    1704 non-null int64
age                1704 non-null object
dtypes: int64(1), object(1)
memory usage: 26.8+ KB


In [100]:
#age.to_excel(path + 'age.xlsx', index=False)

#### Load EDDC_HF_data_filtered_OS.json

In [213]:
fh = open(path + '2_EDDC_HF_data_filtered_OS.json','r')
data = json.load(fh)

In [214]:
for pid in data:
            print(pid, data[pid]['LOS'], data[pid]['30day_readmit'], data[pid]['60day_readmit'], 
                  data[pid]['90day_readmit'], data[pid]['dischargedisposition'])

1000120800270 2 0 0 0 ROUTINE/HOME
1000124200270 4 1 1 1 ROUTINE/HOME
1000155300270 15 0 0 0 ROUTINE/HOME
1000634700270 15 0 0 0 ROUTINE/HOME
1001023900270 9 0 0 0 ROUTINE/HOME
1001176600270 1 0 0 0 DISCHARGE TO OTHER FAC NOT LISTED
1001771000270 2 0 0 0 DISCHARGE HOME WITH HM HLTH SERV
1002721000270 4 0 0 0 DISCHARGE HOME WITH HM HLTH SERV
1002827500270 26 0 0 0 ROUTINE/HOME
1004509300270 3 0 0 0 ROUTINE/HOME
1004549000270 4 0 0 0 ROUTINE/HOME
1005027500270 4 0 0 0 DISCHARGE HOME WITH HM HLTH SERV
1005065000270 12 0 0 0 DISCHARGE TO SNF
1005144400270 2 0 0 0 ROUTINE/HOME
1005518500270 4 0 0 0 DISCHARGE HOME WITH HM HLTH SERV
1005547000270 0 0 0 0 PATIENT EXPIRED
1005619700270 9 0 0 0 DISCHARGE HOME WITH HM HLTH SERV
1005666700270 7 0 0 0 DISCHARGE HOME WITH HM HLTH SERV
1005938300270 2 0 0 0 DISCHARGE TO ACUTE FACILITY
1006137500270 4 0 0 0 DISCHARGE HOME WITH HM HLTH SERV
1006200000270 66 0 0 0 DISCHARGE HOME WITH HM HLTH SERV
1006218700270 3 0 0 0 ROUTINE/HOME
1006778700270 3 0 0 0 

1069295500270 23 0 0 0 DISCHARGE HOSPICE FACILITY
1069508100270 8 0 0 0 ROUTINE/HOME
1069584700270 3 0 0 0 DISCHARGE HOME WITH HM HLTH SERV
1069744600270 4 0 0 0 ROUTINE/HOME
1070402500270 14 0 0 0 DISCHARGE HOME WITH HM HLTH SERV
1070885500270 11 0 0 0 ROUTINE/HOME
1070985600270 11 0 0 0 DISCHARGE HOME WITH HM HLTH SERV
1071165400270 4 0 0 0 AGAINST MEDICAL ADVICE
1071666000270 5 0 0 0 ROUTINE/HOME
1071746300270 8 0 0 0 PATIENT EXPIRED
1072123400270 19 0 0 0 ROUTINE/HOME
1072597800270 3 0 0 0 ROUTINE/HOME
1073180500270 13 0 0 0 DISCHARGE HOME WITH HM HLTH SERV
1073247200270 12 0 0 0 DISCHARGE TO SNF
1075046000270 3 0 0 0 DISCHARGE HOME WITH HM HLTH SERV
1075882300270 2 0 0 0 DISCHARGE HOME WITH HM HLTH SERV
1075947500270 1 0 0 0 ROUTINE/HOME
1075976800270 7 0 0 0 DISCHARGE HOME WITH HM HLTH SERV
1076096200270 51 0 0 0 DISCHARGE TO SNF
1076191500270 2 0 0 0 ROUTINE/HOME
1077020100270 7 0 0 0 ROUTINE/HOME
1077550700270 3 0 0 0 DISCHARGE HOME WITH HM HLTH SERV
1078199600270 6 0 0 0 DISCH

1145234100270 4 0 0 0 DISCHARGE HOME WITH HM HLTH SERV
1145364300270 2 0 0 0 ROUTINE/HOME
1145852600270 6 0 0 0 DISCHARGE HOME WITH HM HLTH SERV
1145898800270 1 0 0 0 ROUTINE/HOME
1146504300270 11 0 0 0 DISCHARGE TO SNF
1146516300270 11 0 0 0 DISCHARGE HOME WITH HM HLTH SERV
1146652100270 6 0 0 0 DISCHARGE HOME WITH HM HLTH SERV
1146735800270 7 0 1 1 DISCHARGE HOME WITH HM HLTH SERV
1146742800270 33 0 0 0 DISCHARGE TO SNF
1146841500270 4 0 0 0 DISCHARGE HOME WITH HM HLTH SERV
1146937800270 5 0 0 0 ROUTINE/HOME
1147096100270 16 0 0 0 DISCHARGE HOME WITH HM HLTH SERV
1147147600270 191 0 0 0 TRANS/DISCHARGE TO/FM IP PSYCH
1147157200270 9 0 0 0 DISCHARGE TO SNF
1147251900270 3 0 0 0 DISCHARGE HOME WITH HM HLTH SERV
1147345500270 5 0 0 0 DISCHARGE TO SNF
1147662100270 22 0 0 0 DISCHARGE HOME WITH HM HLTH SERV
1147679400270 9 0 0 0 ROUTINE/HOME
1147772800270 7 0 0 0 DISCHARGE TO SNF
1147957300270 6 0 0 0 DISCHARGE TO SNF
1148086400270 2 0 0 0 ROUTINE/HOME
1148298400270 3 0 0 0 ROUTINE/HOME
1

1219793500270 1 0 0 0 AGAINST MEDICAL ADVICE
1219856200270 7 0 0 0 ROUTINE/HOME
1219864300270 8 0 0 0 DISCHARGE HOME WITH HM HLTH SERV
1220134000270 7 0 0 0 ROUTINE/HOME
1220441100270 8 0 0 0 DISCHARGE HOME WITH HM HLTH SERV
1221808200270 2 0 0 0 ROUTINE/HOME
1221936300270 5 1 1 1 DISCHARGE HOME WITH HM HLTH SERV
1221988600270 8 0 0 0 ROUTINE/HOME
1222519600270 5 0 0 0 DISCHARGE TO SNF
1223090900270 5 0 0 0 DISCHARGE TO SNF
1223201400270 42 1 1 1 ROUTINE/HOME
1223393300270 9 0 0 0 ROUTINE/HOME
1223874800270 7 0 1 1 DISCHARGE HOME WITH HM HLTH SERV
1224174300270 7 0 0 0 TRANS/DISCHARGE TO/FM IP REHAB FAC
1224183400270 4 0 0 0 DISCHARGE HOME WITH HM HLTH SERV
1224221600270 11 0 0 0 ROUTINE/HOME
1224756700270 13 0 0 0 ROUTINE/HOME
1225166800270 8 0 0 0 ROUTINE/HOME
1225181100270 6 0 0 0 TRANS/DISCHARGE TO/FM IP REHAB FAC
1225771000270 1 0 0 0 DISCHARGE HOME WITH HM HLTH SERV
1226275500270 3 0 0 0 ROUTINE/HOME
1226395400270 17 0 0 0 ROUTINE/HOME
1226438500270 21 0 0 0 DISCHARGE HOME WITH H

820309800270 1 0 0 0 ROUTINE/HOME
820972600270 14 0 0 0 DISCHARGE HOME WITH HM HLTH SERV
822029000270 8 0 0 0 DISCHARGE HOME WITH HM HLTH SERV
822182700270 7 0 0 0 DISCHARGE TO SNF
822184400270 2 0 0 0 DISCHARGE HOME WITH HM HLTH SERV
822570100270 4 0 0 0 ROUTINE/HOME
822646200270 22 0 0 0 ROUTINE/HOME
823415900270 5 0 0 0 ROUTINE/HOME
823750300270 6 0 0 0 PATIENT EXPIRED
823781800270 2 0 0 0 ROUTINE/HOME
824637200270 7 0 0 1 DISCHARGE HOME WITH HM HLTH SERV
824650600270 1 0 0 0 ROUTINE/HOME
824911700270 8 0 0 0 DISCHARGE HOME WITH HM HLTH SERV
825281400270 7 0 0 0 DISCHARGE TO SNF
825655600270 11 0 0 0 DISCHARGE TO SNF
825743400270 11 0 0 0 ROUTINE/HOME
826079200270 2 0 0 0 ROUTINE/HOME
826166100270 2 0 0 0 DISCHARGE HOME WITH HM HLTH SERV
826526100270 2 0 0 0 ROUTINE/HOME
826527800270 6 0 0 0 ROUTINE/HOME
826626700270 9 0 0 0 PATIENT EXPIRED
827067100270 7 0 0 0 DISCHARGE HOME WITH HM HLTH SERV
827266000270 12 0 0 0 DISCHARGE TO SNF
827460000270 5 0 0 0 PATIENT EXPIRED
829344000270 2

898791000270 4 0 0 0 TRANS/DISCHARGE TO/FM IP REHAB FAC
899657100270 1 0 0 0 ROUTINE/HOME
900147800270 18 0 0 0 DISCHARGE HOME WITH HM HLTH SERV
900186500270 32 0 0 0 DISCHARGE TO SNF
900623700270 6 0 0 0 DISCHARGE HOME WITH HM HLTH SERV
900825700270 1 0 0 0 ROUTINE/HOME
900826000270 2 0 0 0 DISCHARGE HOME WITH HM HLTH SERV
901383700270 19 0 0 0 DISCHARGE TO SNF
901757100270 11 0 0 0 DISCHARGE HOME WITH HM HLTH SERV
901791600270 1 0 0 0 ROUTINE/HOME
901839000270 5 0 0 0 ROUTINE/HOME
902232500270 12 0 0 0 ROUTINE/HOME
902810800270 10 0 0 0 TRANS/DISCHARGE TO/FM IP REHAB FAC
902869400270 8 0 0 0 DISCHARGE HOME WITH HM HLTH SERV
902958300270 5 0 1 1 ROUTINE/HOME
903856600270 9 0 0 0 ROUTINE/HOME
904270500270 7 0 0 0 DISCHARGE TO SNF
904337700270 5 0 0 0 DISCHARGE TO SNF
904544500270 7 0 0 0 DISCHARGE TO SNF
904594600270 12 0 0 0 PATIENT EXPIRED
904707500270 5 0 0 0 ROUTINE/HOME
905323500270 3 0 0 0 DISCHARGE HOME WITH HM HLTH SERV
905325700270 5 0 0 0 ROUTINE/HOME
905380100270 5 0 0 0 ROU

965963200270 2 0 0 0 ROUTINE/HOME
966269000270 41 0 0 0 PATIENT EXPIRED
966486500270 5 0 0 0 DISCHARGE TO SNF
966736400270 1 0 0 0 ROUTINE/HOME
967089700270 3 0 0 0 DISCHARGE HOME WITH HM HLTH SERV
967135100270 9 0 0 0 DISCHARGE HOME WITH HOSPICE
967421600270 2 0 0 0 ROUTINE/HOME
967719900270 7 0 0 0 DISCHARGE HOME WITH HM HLTH SERV
968310600270 6 0 0 0 DISCHARGE HOME WITH HM HLTH SERV
968743100270 6 0 1 1 ROUTINE/HOME
969128900270 3 0 0 0 DISCHARGE HOME WITH HM HLTH SERV
969206400270 3 0 0 0 DISCHARGE HOME WITH HM HLTH SERV
969344700270 18 0 0 0 PATIENT EXPIRED
969375500270 9 0 0 0 DISCHARGE HOME WITH HM HLTH SERV
969385600270 4 0 0 0 DISCHARGE HOME WITH HM HLTH SERV
969400600270 10 0 0 0 DISCHARGE TO SNF
969410100270 3 0 0 0 ROUTINE/HOME
969885800270 5 0 0 0 ROUTINE/HOME
970220600270 2 0 0 0 ROUTINE/HOME
970747200270 26 0 1 1 DISCHARGE TO SNF
971002300270 11 0 0 0 DISCHARGE HOME WITH HM HLTH SERV
971472900270 2 0 0 0 DISCHARGE HOME WITH HM HLTH SERV
971793200270 5 0 0 0 DISCHARGE HOM

In [215]:
with open(path + '2_Descriptive_Table.csv', 'w+') as f:
    for pid in data:
            print(pid, data[pid]['LOS'], data[pid]['30day_readmit'], data[pid]['60day_readmit'], 
                  data[pid]['90day_readmit'], data[pid]['dischargedisposition'], file = f, sep = '\t')

In [ ]:
Start here

In [216]:
descriptive = pd.read_csv(path + '2_Descriptive_Table.csv', header=None, sep='\t')
descriptive.columns = ['clientvisitguid', 'LOS', '30day', '60day', '90day', 'dischargedisposition']
descriptive

,clientvisitguid,LOS,30day,60day,90day,dischargedisposition
0,1000120800270,2,0,0,0,ROUTINE/HOME
1,1000124200270,4,1,1,1,ROUTINE/HOME
2,1000155300270,15,0,0,0,ROUTINE/HOME
3,1000634700270,15,0,0,0,ROUTINE/HOME
4,1001023900270,9,0,0,0,ROUTINE/HOME
...,...,...,...,...,...,...
1699,999130900270,3,0,0,0,ROUTINE/HOME
1700,999139900270,1,0,0,0,DISCHARGE HOME WITH HM HLTH SERV
1701,999619900270,11,0,0,0,DISCHARGE TO SNF
1702,999912100270,11,0,0,0,DISCHARGE HOME WITH HM HLTH SERV


In [217]:
descriptive['dischargedisposition'].value_counts()

ROUTINE/HOME                          612
DISCHARGE HOME WITH HM HLTH SERV      602
DISCHARGE TO SNF                      284
PATIENT EXPIRED                        79
TRANS/DISCHARGE TO/FM IP REHAB FAC     35
DISCHARGE HOME WITH HOSPICE            23
DISCHARGE HOSPICE FACILITY             22
AGAINST MEDICAL ADVICE                 15
DISCHARGE TO ACUTE FACILITY             9
DISCHARGE TO OTHER FAC NOT LISTED       5
DISCHARGE TO INTERIM FACILITY           4
DISCHARGE TO LAW ENFORCEMENT            2
DISCHARGE TO LONG TERM CARE             2
TRANS/DISCHARGE TO/FM IP PSYCH          1
Name: dischargedisposition, dtype: int64

In [218]:
descriptive['clientvisitguid'] = descriptive['clientvisitguid'].astype(int)
descriptive['clientvisitguid'].nunique()

1704

In [219]:
descriptive['home'] = descriptive.dischargedisposition.apply(lambda x: 1 if x == 'ROUTINE/HOME' else 0)
descriptive['death'] = descriptive.dischargedisposition.apply(lambda x: 1 if x == 'PATIENT EXPIRED' else 0)

In [220]:
descriptive['home'].value_counts()

0    1092
1     612
Name: home, dtype: int64

In [221]:
descriptive['death'].value_counts()

0    1625
1      79
Name: death, dtype: int64

In [222]:
descriptive

,clientvisitguid,LOS,30day,60day,90day,dischargedisposition,home,death
0,1000120800270,2,0,0,0,ROUTINE/HOME,1,0
1,1000124200270,4,1,1,1,ROUTINE/HOME,1,0
2,1000155300270,15,0,0,0,ROUTINE/HOME,1,0
3,1000634700270,15,0,0,0,ROUTINE/HOME,1,0
4,1001023900270,9,0,0,0,ROUTINE/HOME,1,0
...,...,...,...,...,...,...,...,...
1699,999130900270,3,0,0,0,ROUTINE/HOME,1,0
1700,999139900270,1,0,0,0,DISCHARGE HOME WITH HM HLTH SERV,0,0
1701,999619900270,11,0,0,0,DISCHARGE TO SNF,0,0
1702,999912100270,11,0,0,0,DISCHARGE HOME WITH HM HLTH SERV,0,0


### (Merge train_cluster.csv)

In [25]:
descriptive = pd.merge(left = descriptive, right = train_cluster, 
                       left_on = 'clientvisitguid', right_on = 'clientvisitguid', how = 'right')
descriptive

,clientvisitguid,LOS,30day,60day,90day,dischargedisposition,home,death,cluster_by30
0,1000120800270,2,0,0,0,ROUTINE/HOME,1,0,1
1,1000155300270,15,0,0,0,ROUTINE/HOME,1,0,1
2,1000634700270,15,0,0,0,ROUTINE/HOME,1,0,2
3,1001176600270,1,0,0,0,DISCHARGE TO OTHER FAC NOT LISTED,0,0,1
4,1001771000270,2,0,0,0,DISCHARGE HOME WITH HM HLTH SERV,0,0,1
...,...,...,...,...,...,...,...,...,...
1379,999130900270,3,0,0,0,ROUTINE/HOME,1,0,1
1380,999139900270,1,0,0,0,DISCHARGE HOME WITH HM HLTH SERV,0,0,1
1381,999619900270,11,0,0,0,DISCHARGE TO SNF,0,0,1
1382,999912100270,11,0,0,0,DISCHARGE HOME WITH HM HLTH SERV,0,0,1


### Merge comorbidity score

In [223]:
descriptive_score = pd.merge(left = descriptive, right = comorbidityscore, 
                       left_on = 'clientvisitguid', right_on = 'ID', how = 'left').drop(['ID'], axis='columns')
descriptive_score

,clientvisitguid,LOS,30day,60day,90day,dischargedisposition,home,death,Elixhauser,Charlson
0,1000120800270,2,0,0,0,ROUTINE/HOME,1,0,4,4
1,1000124200270,4,1,1,1,ROUTINE/HOME,1,0,4,2
2,1000155300270,15,0,0,0,ROUTINE/HOME,1,0,3,1
3,1000634700270,15,0,0,0,ROUTINE/HOME,1,0,3,1
4,1001023900270,9,0,0,0,ROUTINE/HOME,1,0,2,1
...,...,...,...,...,...,...,...,...,...,...
1699,999130900270,3,0,0,0,ROUTINE/HOME,1,0,2,2
1700,999139900270,1,0,0,0,DISCHARGE HOME WITH HM HLTH SERV,0,0,3,2
1701,999619900270,11,0,0,0,DISCHARGE TO SNF,0,0,4,2
1702,999912100270,11,0,0,0,DISCHARGE HOME WITH HM HLTH SERV,0,0,1,1


### Merge userQ

In [224]:
descriptive_userq = pd.merge(left = descriptive_score, right = userq, 
                       left_on = 'clientvisitguid', right_on = 'clientvisitguid', how = 'left')
descriptive_userq

,clientvisitguid,LOS,30day,60day,90day,dischargedisposition,home,death,Elixhauser,Charlson,userQ
0,1000120800270,2,0,0,0,ROUTINE/HOME,1,0,4,4,Q1
1,1000124200270,4,1,1,1,ROUTINE/HOME,1,0,4,2,Q1
2,1000155300270,15,0,0,0,ROUTINE/HOME,1,0,3,1,Q1
3,1000634700270,15,0,0,0,ROUTINE/HOME,1,0,3,1,Q3
4,1001023900270,9,0,0,0,ROUTINE/HOME,1,0,2,1,Q1
...,...,...,...,...,...,...,...,...,...,...,...
1699,999130900270,3,0,0,0,ROUTINE/HOME,1,0,2,2,NaN
1700,999139900270,1,0,0,0,DISCHARGE HOME WITH HM HLTH SERV,0,0,3,2,Q1
1701,999619900270,11,0,0,0,DISCHARGE TO SNF,0,0,4,2,NaN
1702,999912100270,11,0,0,0,DISCHARGE HOME WITH HM HLTH SERV,0,0,1,1,Q1


In [225]:
descriptive_userq[descriptive_userq['clientvisitguid']==685526500270]

,clientvisitguid,LOS,30day,60day,90day,dischargedisposition,home,death,Elixhauser,Charlson,userQ
972,685526500270,10,0,0,0,ROUTINE/HOME,1,0,2,2,NaN


In [47]:
# skip
for i in range(len(descriptive['userQ'])):   ## userQ is the column 10
    if descriptive.iloc[i, 10] == 'Q1':
        descriptive.iloc[i, 10] = 'low'
    elif descriptive.iloc[i, 10] == 'Q3':
        descriptive.iloc[i, 10] = 'high'
    else:
        descriptive.iloc[i, 10] = 'medium'   
descriptive

,clientvisitguid,LOS,30day,60day,90day,dischargedisposition,home,death,Elixhauser,Charlson,userQ
0,1000120800270,2,0,0,0,ROUTINE/HOME,1,0,4,4,low
1,1000124200270,4,1,1,1,ROUTINE/HOME,1,0,4,2,low
2,1000155300270,15,0,0,0,ROUTINE/HOME,1,0,3,1,low
3,1000634700270,15,0,0,0,ROUTINE/HOME,1,0,3,1,high
4,1001023900270,9,0,0,0,ROUTINE/HOME,1,0,2,1,low
...,...,...,...,...,...,...,...,...,...,...,...
1699,999130900270,3,0,0,0,ROUTINE/HOME,1,0,2,2,medium
1700,999139900270,1,0,0,0,DISCHARGE HOME WITH HM HLTH SERV,0,0,3,2,low
1701,999619900270,11,0,0,0,DISCHARGE TO SNF,0,0,4,2,medium
1702,999912100270,11,0,0,0,DISCHARGE HOME WITH HM HLTH SERV,0,0,1,1,low


### Merge CCW

In [226]:
descriptive_ccw = pd.merge(left = descriptive_userq, right = ccw, 
                       left_on = 'clientvisitguid', right_on = 'clientvisitguid', how = 'left')
descriptive_ccw

,clientvisitguid,LOS,30day,60day,90day,dischargedisposition,home,death,Elixhauser,Charlson,userQ,CCW
0,1000120800270,2,0,0,0,ROUTINE/HOME,1,0,4,4,Q1,3
1,1000124200270,4,1,1,1,ROUTINE/HOME,1,0,4,2,Q1,2
2,1000155300270,15,0,0,0,ROUTINE/HOME,1,0,3,1,Q1,1
3,1000634700270,15,0,0,0,ROUTINE/HOME,1,0,3,1,Q3,3
4,1001023900270,9,0,0,0,ROUTINE/HOME,1,0,2,1,Q1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1699,999130900270,3,0,0,0,ROUTINE/HOME,1,0,2,2,NaN,0
1700,999139900270,1,0,0,0,DISCHARGE HOME WITH HM HLTH SERV,0,0,3,2,Q1,2
1701,999619900270,11,0,0,0,DISCHARGE TO SNF,0,0,4,2,NaN,2
1702,999912100270,11,0,0,0,DISCHARGE HOME WITH HM HLTH SERV,0,0,1,1,Q1,1


### Merge VTE

In [227]:
descriptive_vte = pd.merge(left = descriptive_ccw, right = vte, 
                       left_on = 'clientvisitguid', right_on = 'clientvisitguid', how = 'left')
descriptive_vte

,clientvisitguid,LOS,30day,60day,90day,dischargedisposition,home,death,Elixhauser,Charlson,userQ,CCW,VTE
0,1000120800270,2,0,0,0,ROUTINE/HOME,1,0,4,4,Q1,3,0
1,1000124200270,4,1,1,1,ROUTINE/HOME,1,0,4,2,Q1,2,0
2,1000155300270,15,0,0,0,ROUTINE/HOME,1,0,3,1,Q1,1,0
3,1000634700270,15,0,0,0,ROUTINE/HOME,1,0,3,1,Q3,3,0
4,1001023900270,9,0,0,0,ROUTINE/HOME,1,0,2,1,Q1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1699,999130900270,3,0,0,0,ROUTINE/HOME,1,0,2,2,NaN,0,0
1700,999139900270,1,0,0,0,DISCHARGE HOME WITH HM HLTH SERV,0,0,3,2,Q1,2,0
1701,999619900270,11,0,0,0,DISCHARGE TO SNF,0,0,4,2,NaN,2,0
1702,999912100270,11,0,0,0,DISCHARGE HOME WITH HM HLTH SERV,0,0,1,1,Q1,1,0


### Merge age

In [228]:
descriptive_age = pd.merge(left = descriptive_vte, right = age, 
                       left_on = 'clientvisitguid', right_on = 'clientvisitguid', how = 'left')
descriptive_age

,clientvisitguid,LOS,30day,60day,90day,dischargedisposition,home,death,Elixhauser,Charlson,userQ,CCW,VTE,age
0,1000120800270,2,0,0,0,ROUTINE/HOME,1,0,4,4,Q1,3,0,61
1,1000124200270,4,1,1,1,ROUTINE/HOME,1,0,4,2,Q1,2,0,58
2,1000155300270,15,0,0,0,ROUTINE/HOME,1,0,3,1,Q1,1,0,66
3,1000634700270,15,0,0,0,ROUTINE/HOME,1,0,3,1,Q3,3,0,77
4,1001023900270,9,0,0,0,ROUTINE/HOME,1,0,2,1,Q1,1,0,80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1699,999130900270,3,0,0,0,ROUTINE/HOME,1,0,2,2,NaN,0,0,41
1700,999139900270,1,0,0,0,DISCHARGE HOME WITH HM HLTH SERV,0,0,3,2,Q1,2,0,85
1701,999619900270,11,0,0,0,DISCHARGE TO SNF,0,0,4,2,NaN,2,0,88
1702,999912100270,11,0,0,0,DISCHARGE HOME WITH HM HLTH SERV,0,0,1,1,Q1,1,0,74


In [229]:
descriptive_age[descriptive_age['clientvisitguid']==1000155300270]

,clientvisitguid,LOS,30day,60day,90day,dischargedisposition,home,death,Elixhauser,Charlson,userQ,CCW,VTE,age
2,1000155300270,15,0,0,0,ROUTINE/HOME,1,0,3,1,Q1,1,0,66


### Merge clientvisit

In [65]:
clientvisit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14044 entries, 0 to 14043
Data columns (total 9 columns):
DischargeDCDesc         13640 non-null object
DischargeDtm            13580 non-null object
AdmitDtm                14044 non-null object
ClientVisitGUID         14044 non-null int64
ClientGUID              14044 non-null int64
HospitalService         13454 non-null object
EDSite                  11136 non-null object
SummaryDispoCategory    11136 non-null object
FinalDispoCategory      11136 non-null object
dtypes: int64(2), object(7)
memory usage: 987.6+ KB


In [230]:
descriptive_visit = pd.merge(left = descriptive_age, right = clientvisit, 
                       left_on = 'clientvisitguid', right_on = 'ClientVisitGUID', how = 'left')
descriptive_visit

,clientvisitguid,LOS,30day,60day,90day,dischargedisposition,home,death,Elixhauser,Charlson,...,age,DischargeDCDesc,DischargeDtm,AdmitDtm,ClientVisitGUID,ClientGUID,HospitalService,EDSite,SummaryDispoCategory,FinalDispoCategory
0,1000120800270,2,0,0,0,ROUTINE/HOME,1,0,4,4,...,61,ROUTINE/HOME,2017-04-18 15:08:00.000,2017-04-16 15:14:00.000,1000120800270,173577400200,MED,LMH ED,Admitted,Admitted
1,1000124200270,4,1,1,1,ROUTINE/HOME,1,0,4,2,...,58,ROUTINE/HOME,2017-04-21 12:30:00.000,2017-04-16 14:55:00.000,1000124200270,224446700200,MED,LMH ED,Admitted,Admitted
2,1000155300270,15,0,0,0,ROUTINE/HOME,1,0,3,1,...,66,ROUTINE/HOME,2017-05-02 12:37:00.000,2017-04-17 02:37:00.000,1000155300270,244485700200,MED,Cornell ED,Admitted,Admitted
3,1000634700270,15,0,0,0,ROUTINE/HOME,1,0,3,1,...,77,ROUTINE/HOME,2017-05-03 14:07:00.000,2017-04-17 23:13:00.000,1000634700270,242406300200,MED,LMH ED,Admitted,Admitted
4,1001023900270,9,0,0,0,ROUTINE/HOME,1,0,2,1,...,80,ROUTINE/HOME,2017-04-28 11:00:00.000,2017-04-18 22:08:00.000,1001023900270,31947500200,MED,Cornell ED,Admitted,Admitted
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1702,999130900270,3,0,0,0,ROUTINE/HOME,1,0,2,2,...,41,ROUTINE/HOME,2017-04-16 14:08:00.000,2017-04-13 03:53:00.000,999130900270,1394500200,MED,Cornell ED,Admitted,Admitted
1703,999139900270,1,0,0,0,DISCHARGE HOME WITH HM HLTH SERV,0,0,3,2,...,85,DISCHARGE HOME WITH HM HLTH SERV,2017-04-14 13:00:00.000,2017-04-13 02:51:00.000,999139900270,10446600200,MED,Cornell ED,Admitted,Admitted
1704,999619900270,11,0,0,0,DISCHARGE TO SNF,0,0,4,2,...,88,DISCHARGE TO SNF,2017-04-25 15:20:00.000,2017-04-13 22:47:00.000,999619900270,249928800200,MED,LMH ED,Admitted,Admitted
1705,999912100270,11,0,0,0,DISCHARGE HOME WITH HM HLTH SERV,0,0,1,1,...,74,DISCHARGE HOME WITH HM HLTH SERV,2017-04-26 19:18:00.000,2017-04-14 21:35:00.000,999912100270,73268000200,MED,Cornell ED,Admitted,Admitted


In [231]:
descriptive_visit = descriptive_visit.drop(['DischargeDCDesc', 'DischargeDtm', 'AdmitDtm', 'ClientVisitGUID', 'ClientGUID', 
                                'HospitalService', 'SummaryDispoCategory'], axis = 'columns')

In [232]:
## there're more rows
descriptive_visit[descriptive_visit.duplicated('clientvisitguid')==True]

,clientvisitguid,LOS,30day,60day,90day,dischargedisposition,home,death,Elixhauser,Charlson,userQ,CCW,VTE,age,EDSite,FinalDispoCategory
176,1049056200270,19,0,0,0,DISCHARGE TO LONG TERM CARE,0,0,5,2,NaN,0,0,93,Cornell ED,Admitted
851,1249612100270,3,0,0,0,ROUTINE/HOME,1,0,2,1,Q1,0,0,70,LMH ED,Admitted
969,608150700270,13,0,0,0,DISCHARGE TO LONG TERM CARE,0,0,3,1,NaN,1,0,71,LMH ED,Admitted


In [234]:
descriptive_visit[descriptive_visit['clientvisitguid'] == 1049056200270]

,clientvisitguid,LOS,30day,60day,90day,dischargedisposition,home,death,Elixhauser,Charlson,userQ,CCW,VTE,age,EDSite,FinalDispoCategory
175,1049056200270,19,0,0,0,DISCHARGE TO LONG TERM CARE,0,0,5,2,NaN,0,0,93,Cornell ED,Admitted
176,1049056200270,19,0,0,0,DISCHARGE TO LONG TERM CARE,0,0,5,2,NaN,0,0,93,Cornell ED,Admitted


In [235]:
## in this way, I can drop the duplicates directly
descriptive_visit.drop_duplicates(subset = 'clientvisitguid', keep = 'first', inplace = True) 

In [71]:
resultlist_ap=pd.read_csv(path+'2_visit_EDDC_2012_2018.csv')

In [74]:
#descriptive_ap = pd.merge(left = descriptive_age, right = resultlist_ap, 
#                       left_on = 'clientvisitguid', right_on = 'clientvisitguid', how = 'left')
#descriptive_ap

,clientvisitguid,LOS,30day,60day,90day,dischargedisposition_x,home,death,Elixhauser,Charlson,...,VTE,age,dischargedisposition_y,dischargedtm,admitdtm,clientguid,service,EDSite,SummaryDispoCategory,FinalDispoCategory
0,1000120800270,2,0,0,0,ROUTINE/HOME,1,0,4,4,...,0,61,ROUTINE/HOME,2017-04-18 15:00:00,2017-04-16 15:00:00,173577400200,MED,LMH ED,Admitted,Admitted
1,1000124200270,4,1,1,1,ROUTINE/HOME,1,0,4,2,...,0,58,ROUTINE/HOME,2017-04-21 12:00:00,2017-04-16 14:00:00,224446700200,MED,LMH ED,Admitted,Admitted
2,1000155300270,15,0,0,0,ROUTINE/HOME,1,0,3,1,...,0,66,ROUTINE/HOME,2017-05-02 12:00:00,2017-04-17 02:00:00,244485700200,MED,Cornell ED,Admitted,Admitted
3,1000634700270,15,0,0,0,ROUTINE/HOME,1,0,3,1,...,0,77,ROUTINE/HOME,2017-05-03 14:00:00,2017-04-17 23:00:00,242406300200,MED,LMH ED,Admitted,Admitted
4,1001023900270,9,0,0,0,ROUTINE/HOME,1,0,2,1,...,0,80,ROUTINE/HOME,2017-04-28 11:00:00,2017-04-18 22:00:00,31947500200,MED,Cornell ED,Admitted,Admitted
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1702,999130900270,3,0,0,0,ROUTINE/HOME,1,0,2,2,...,0,41,ROUTINE/HOME,2017-04-16 14:00:00,2017-04-13 03:00:00,1394500200,MED,Cornell ED,Admitted,Admitted
1703,999139900270,1,0,0,0,DISCHARGE HOME WITH HM HLTH SERV,0,0,3,2,...,0,85,DISCHARGE HOME WITH HM HLTH SERV,2017-04-14 13:00:00,2017-04-13 02:00:00,10446600200,MED,Cornell ED,Admitted,Admitted
1704,999619900270,11,0,0,0,DISCHARGE TO SNF,0,0,4,2,...,0,88,DISCHARGE TO SNF,2017-04-25 15:00:00,2017-04-13 22:00:00,249928800200,MED,LMH ED,Admitted,Admitted
1705,999912100270,11,0,0,0,DISCHARGE HOME WITH HM HLTH SERV,0,0,1,1,...,0,74,DISCHARGE HOME WITH HM HLTH SERV,2017-04-26 19:00:00,2017-04-14 21:00:00,73268000200,MED,Cornell ED,Admitted,Admitted


In [75]:
#descriptive_ap[descriptive_ap['clientvisitguid']==608150700270]

,clientvisitguid,LOS,30day,60day,90day,dischargedisposition_x,home,death,Elixhauser,Charlson,...,VTE,age,dischargedisposition_y,dischargedtm,admitdtm,clientguid,service,EDSite,SummaryDispoCategory,FinalDispoCategory
968,608150700270,13,0,0,0,DISCHARGE TO LONG TERM CARE,0,0,3,1,...,0,71,ADMITTED TO SAME FACILITY,2014-05-16 15:00:00,2014-05-02 23:00:00,158264000200,MED,LMH ED,Admitted,Admitted
969,608150700270,13,0,0,0,DISCHARGE TO LONG TERM CARE,0,0,3,1,...,0,71,DISCHARGE TO LONG TERM CARE,2014-05-16 15:00:00,2014-05-02 23:00:00,158264000200,MED,LMH ED,Admitted,Admitted


In [236]:
descriptive_visit['campus'] = descriptive_visit['EDSite'].str.strip(' ED')

In [237]:
descriptive_visit

,clientvisitguid,LOS,30day,60day,90day,dischargedisposition,home,death,Elixhauser,Charlson,userQ,CCW,VTE,age,EDSite,FinalDispoCategory,campus
0,1000120800270,2,0,0,0,ROUTINE/HOME,1,0,4,4,Q1,3,0,61,LMH ED,Admitted,LMH
1,1000124200270,4,1,1,1,ROUTINE/HOME,1,0,4,2,Q1,2,0,58,LMH ED,Admitted,LMH
2,1000155300270,15,0,0,0,ROUTINE/HOME,1,0,3,1,Q1,1,0,66,Cornell ED,Admitted,Cornell
3,1000634700270,15,0,0,0,ROUTINE/HOME,1,0,3,1,Q3,3,0,77,LMH ED,Admitted,LMH
4,1001023900270,9,0,0,0,ROUTINE/HOME,1,0,2,1,Q1,1,0,80,Cornell ED,Admitted,Cornell
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1702,999130900270,3,0,0,0,ROUTINE/HOME,1,0,2,2,NaN,0,0,41,Cornell ED,Admitted,Cornell
1703,999139900270,1,0,0,0,DISCHARGE HOME WITH HM HLTH SERV,0,0,3,2,Q1,2,0,85,Cornell ED,Admitted,Cornell
1704,999619900270,11,0,0,0,DISCHARGE TO SNF,0,0,4,2,NaN,2,0,88,LMH ED,Admitted,LMH
1705,999912100270,11,0,0,0,DISCHARGE HOME WITH HM HLTH SERV,0,0,1,1,Q1,1,0,74,Cornell ED,Admitted,Cornell


### Merge demographics

In [238]:
descriptive_demo = pd.merge(left = descriptive_visit, right = demo, 
                       left_on = 'clientvisitguid', right_on = 'clientvisitguid', how = 'left')

In [239]:
descriptive_demo

,clientvisitguid,LOS,30day,60day,90day,dischargedisposition,home,death,Elixhauser,Charlson,...,FinalDispoCategory,campus,clientguid,ethnicity,Ethnicity2,BirthYearNum,LanguageCode,GenderCode,RaceCode,MaritalStatusCode
0,1000120800270,2,0,0,0,ROUTINE/HOME,1,0,4,4,...,Admitted,LMH,173577400200,U,U,1957,English,Male,Other Not Described,Single
1,1000124200270,4,1,1,1,ROUTINE/HOME,1,0,4,2,...,Admitted,LMH,224446700200,N,NaN,1960,English,Male,Black/African Am.,Single
2,1000155300270,15,0,0,0,ROUTINE/HOME,1,0,3,1,...,Admitted,Cornell,244485700200,N,N,1952,English,Male,White,Single
3,1000634700270,15,0,0,0,ROUTINE/HOME,1,0,3,1,...,Admitted,LMH,242406300200,N,NaN,1941,English,Male,White,Single
4,1001023900270,9,0,0,0,ROUTINE/HOME,1,0,2,1,...,Admitted,Cornell,31947500200,N,N,1938,ENG,Male,W,W
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1699,999130900270,3,0,0,0,ROUTINE/HOME,1,0,2,2,...,Admitted,Cornell,1394500200,HM,HM,1977,English,Female,Other Not Described,Single
1700,999139900270,1,0,0,0,DISCHARGE HOME WITH HM HLTH SERV,0,0,3,2,...,Admitted,Cornell,10446600200,U,NaN,1933,English,Female,Declined,Widower
1701,999619900270,11,0,0,0,DISCHARGE TO SNF,0,0,4,2,...,Admitted,LMH,249928800200,N,NaN,1930,Cantonese,Female,Asian,Single
1702,999912100270,11,0,0,0,DISCHARGE HOME WITH HM HLTH SERV,0,0,1,1,...,Admitted,Cornell,73268000200,N,N,1944,English,Male,Black/African Am.,Single


In [240]:
descriptive_demo = descriptive_demo.drop(['clientguid', 'Ethnicity2', 'BirthYearNum'], axis = 'columns')
descriptive_demo

,clientvisitguid,LOS,30day,60day,90day,dischargedisposition,home,death,Elixhauser,Charlson,...,VTE,age,EDSite,FinalDispoCategory,campus,ethnicity,LanguageCode,GenderCode,RaceCode,MaritalStatusCode
0,1000120800270,2,0,0,0,ROUTINE/HOME,1,0,4,4,...,0,61,LMH ED,Admitted,LMH,U,English,Male,Other Not Described,Single
1,1000124200270,4,1,1,1,ROUTINE/HOME,1,0,4,2,...,0,58,LMH ED,Admitted,LMH,N,English,Male,Black/African Am.,Single
2,1000155300270,15,0,0,0,ROUTINE/HOME,1,0,3,1,...,0,66,Cornell ED,Admitted,Cornell,N,English,Male,White,Single
3,1000634700270,15,0,0,0,ROUTINE/HOME,1,0,3,1,...,0,77,LMH ED,Admitted,LMH,N,English,Male,White,Single
4,1001023900270,9,0,0,0,ROUTINE/HOME,1,0,2,1,...,0,80,Cornell ED,Admitted,Cornell,N,ENG,Male,W,W
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1699,999130900270,3,0,0,0,ROUTINE/HOME,1,0,2,2,...,0,41,Cornell ED,Admitted,Cornell,HM,English,Female,Other Not Described,Single
1700,999139900270,1,0,0,0,DISCHARGE HOME WITH HM HLTH SERV,0,0,3,2,...,0,85,Cornell ED,Admitted,Cornell,U,English,Female,Declined,Widower
1701,999619900270,11,0,0,0,DISCHARGE TO SNF,0,0,4,2,...,0,88,LMH ED,Admitted,LMH,N,Cantonese,Female,Asian,Single
1702,999912100270,11,0,0,0,DISCHARGE HOME WITH HM HLTH SERV,0,0,1,1,...,0,74,Cornell ED,Admitted,Cornell,N,English,Male,Black/African Am.,Single


In [241]:
descriptive_demo.columns

Index(['clientvisitguid', 'LOS', '30day', '60day', '90day',
       'dischargedisposition', 'home', 'death', 'Elixhauser', 'Charlson',
       'userQ', 'CCW', 'VTE', 'age', 'EDSite', 'FinalDispoCategory', 'campus',
       'ethnicity', 'LanguageCode', 'GenderCode', 'RaceCode',
       'MaritalStatusCode'],
      dtype='object')

In [242]:
descriptive_demo['ethnicity_c'] = descriptive_demo['ethnicity']
descriptive_demo

,clientvisitguid,LOS,30day,60day,90day,dischargedisposition,home,death,Elixhauser,Charlson,...,age,EDSite,FinalDispoCategory,campus,ethnicity,LanguageCode,GenderCode,RaceCode,MaritalStatusCode,ethnicity_c
0,1000120800270,2,0,0,0,ROUTINE/HOME,1,0,4,4,...,61,LMH ED,Admitted,LMH,U,English,Male,Other Not Described,Single,U
1,1000124200270,4,1,1,1,ROUTINE/HOME,1,0,4,2,...,58,LMH ED,Admitted,LMH,N,English,Male,Black/African Am.,Single,N
2,1000155300270,15,0,0,0,ROUTINE/HOME,1,0,3,1,...,66,Cornell ED,Admitted,Cornell,N,English,Male,White,Single,N
3,1000634700270,15,0,0,0,ROUTINE/HOME,1,0,3,1,...,77,LMH ED,Admitted,LMH,N,English,Male,White,Single,N
4,1001023900270,9,0,0,0,ROUTINE/HOME,1,0,2,1,...,80,Cornell ED,Admitted,Cornell,N,ENG,Male,W,W,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1699,999130900270,3,0,0,0,ROUTINE/HOME,1,0,2,2,...,41,Cornell ED,Admitted,Cornell,HM,English,Female,Other Not Described,Single,HM
1700,999139900270,1,0,0,0,DISCHARGE HOME WITH HM HLTH SERV,0,0,3,2,...,85,Cornell ED,Admitted,Cornell,U,English,Female,Declined,Widower,U
1701,999619900270,11,0,0,0,DISCHARGE TO SNF,0,0,4,2,...,88,LMH ED,Admitted,LMH,N,Cantonese,Female,Asian,Single,N
1702,999912100270,11,0,0,0,DISCHARGE HOME WITH HM HLTH SERV,0,0,1,1,...,74,Cornell ED,Admitted,Cornell,N,English,Male,Black/African Am.,Single,N


In [243]:
descriptive_demo['ethnicity'].value_counts()

N     1045
U      334
H      201
D       62
HM      38
SS       5
HO       4
SJ       4
HA       2
HD       2
HE       2
HJ       1
SK       1
SI       1
SA       1
ST       1
Name: ethnicity, dtype: int64

In [244]:
descriptive_demo[descriptive_demo['clientvisitguid']==1269084300270]

,clientvisitguid,LOS,30day,60day,90day,dischargedisposition,home,death,Elixhauser,Charlson,...,age,EDSite,FinalDispoCategory,campus,ethnicity,LanguageCode,GenderCode,RaceCode,MaritalStatusCode,ethnicity_c
924,1269084300270,19,0,0,0,DISCHARGE HOME WITH HM HLTH SERV,0,0,3,1,...,80,NaN,NaN,NaN,N,English,Male,White,Married,N


In [245]:
for i in range(len(descriptive_demo['clientvisitguid'])):   ## ethnicity is the column 17, ethnicity_c is the column 22
    if (descriptive_demo.iloc[i, 17] == 'N'):  ## yiye take nan as N
        descriptive_demo.iloc[i, 22] = 'N'
    elif (descriptive_demo.iloc[i, 17] == 'U') or (descriptive_demo.iloc[i, 17] == 'D') or (descriptive_demo.iloc[i, 17] == 'nan'):
        descriptive_demo.iloc[i, 22] = 'unknown'
    else:
        descriptive_demo.iloc[i, 22] = 'H' 

In [246]:
descriptive_demo

,clientvisitguid,LOS,30day,60day,90day,dischargedisposition,home,death,Elixhauser,Charlson,...,age,EDSite,FinalDispoCategory,campus,ethnicity,LanguageCode,GenderCode,RaceCode,MaritalStatusCode,ethnicity_c
0,1000120800270,2,0,0,0,ROUTINE/HOME,1,0,4,4,...,61,LMH ED,Admitted,LMH,U,English,Male,Other Not Described,Single,unknown
1,1000124200270,4,1,1,1,ROUTINE/HOME,1,0,4,2,...,58,LMH ED,Admitted,LMH,N,English,Male,Black/African Am.,Single,N
2,1000155300270,15,0,0,0,ROUTINE/HOME,1,0,3,1,...,66,Cornell ED,Admitted,Cornell,N,English,Male,White,Single,N
3,1000634700270,15,0,0,0,ROUTINE/HOME,1,0,3,1,...,77,LMH ED,Admitted,LMH,N,English,Male,White,Single,N
4,1001023900270,9,0,0,0,ROUTINE/HOME,1,0,2,1,...,80,Cornell ED,Admitted,Cornell,N,ENG,Male,W,W,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1699,999130900270,3,0,0,0,ROUTINE/HOME,1,0,2,2,...,41,Cornell ED,Admitted,Cornell,HM,English,Female,Other Not Described,Single,H
1700,999139900270,1,0,0,0,DISCHARGE HOME WITH HM HLTH SERV,0,0,3,2,...,85,Cornell ED,Admitted,Cornell,U,English,Female,Declined,Widower,unknown
1701,999619900270,11,0,0,0,DISCHARGE TO SNF,0,0,4,2,...,88,LMH ED,Admitted,LMH,N,Cantonese,Female,Asian,Single,N
1702,999912100270,11,0,0,0,DISCHARGE HOME WITH HM HLTH SERV,0,0,1,1,...,74,Cornell ED,Admitted,Cornell,N,English,Male,Black/African Am.,Single,N


In [247]:
descriptive_demo['language_c'] = descriptive_demo['LanguageCode']

In [248]:
descriptive_demo['LanguageCode'].value_counts()

English       1235
Spanish        129
Cantonese       85
ENG             71
Mandarin        38
Chinese         33
Russian         28
Unknown         13
SPA             10
Other           10
Arabic           7
CAN              7
Korean           6
Italian          5
Albanian         5
French           3
Japanese         2
GRE              2
Greek            2
Ot               1
Bengali          1
German           1
Polish           1
Urdu             1
MAN              1
Thai             1
Vietnamese       1
Serbian          1
TUR              1
FRE              1
RUS              1
Romanian         1
Name: LanguageCode, dtype: int64

In [249]:
for i in range(len(descriptive_demo['clientvisitguid'])):   ## LanguageCode is the column 18, language_c is the column 23
    if (descriptive_demo.iloc[i, 18] == 'English') or (descriptive_demo.iloc[i, 18] == 'ENG'):
        descriptive_demo.iloc[i, 23] = 'English'
    else:
        descriptive_demo.iloc[i, 23] = 'Non-English' 

In [250]:
descriptive_demo

,clientvisitguid,LOS,30day,60day,90day,dischargedisposition,home,death,Elixhauser,Charlson,...,EDSite,FinalDispoCategory,campus,ethnicity,LanguageCode,GenderCode,RaceCode,MaritalStatusCode,ethnicity_c,language_c
0,1000120800270,2,0,0,0,ROUTINE/HOME,1,0,4,4,...,LMH ED,Admitted,LMH,U,English,Male,Other Not Described,Single,unknown,English
1,1000124200270,4,1,1,1,ROUTINE/HOME,1,0,4,2,...,LMH ED,Admitted,LMH,N,English,Male,Black/African Am.,Single,N,English
2,1000155300270,15,0,0,0,ROUTINE/HOME,1,0,3,1,...,Cornell ED,Admitted,Cornell,N,English,Male,White,Single,N,English
3,1000634700270,15,0,0,0,ROUTINE/HOME,1,0,3,1,...,LMH ED,Admitted,LMH,N,English,Male,White,Single,N,English
4,1001023900270,9,0,0,0,ROUTINE/HOME,1,0,2,1,...,Cornell ED,Admitted,Cornell,N,ENG,Male,W,W,N,English
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1699,999130900270,3,0,0,0,ROUTINE/HOME,1,0,2,2,...,Cornell ED,Admitted,Cornell,HM,English,Female,Other Not Described,Single,H,English
1700,999139900270,1,0,0,0,DISCHARGE HOME WITH HM HLTH SERV,0,0,3,2,...,Cornell ED,Admitted,Cornell,U,English,Female,Declined,Widower,unknown,English
1701,999619900270,11,0,0,0,DISCHARGE TO SNF,0,0,4,2,...,LMH ED,Admitted,LMH,N,Cantonese,Female,Asian,Single,N,Non-English
1702,999912100270,11,0,0,0,DISCHARGE HOME WITH HM HLTH SERV,0,0,1,1,...,Cornell ED,Admitted,Cornell,N,English,Male,Black/African Am.,Single,N,English


In [251]:
descriptive_demo['gender_c'] = descriptive_demo['GenderCode']

In [252]:
descriptive_demo['GenderCode'].value_counts()

Male      950
Female    752
F           2
Name: GenderCode, dtype: int64

In [253]:
for i in range(len(descriptive_demo['GenderCode'])):   ## GenderCode is the column 19, gender_c is the column 24
    if descriptive_demo.iloc[i, 19] == 'Male':
        descriptive_demo.iloc[i, 24] = 'Male'
    else:
        descriptive_demo.iloc[i, 19] = 'Female'   

In [254]:
descriptive_demo

,clientvisitguid,LOS,30day,60day,90day,dischargedisposition,home,death,Elixhauser,Charlson,...,FinalDispoCategory,campus,ethnicity,LanguageCode,GenderCode,RaceCode,MaritalStatusCode,ethnicity_c,language_c,gender_c
0,1000120800270,2,0,0,0,ROUTINE/HOME,1,0,4,4,...,Admitted,LMH,U,English,Male,Other Not Described,Single,unknown,English,Male
1,1000124200270,4,1,1,1,ROUTINE/HOME,1,0,4,2,...,Admitted,LMH,N,English,Male,Black/African Am.,Single,N,English,Male
2,1000155300270,15,0,0,0,ROUTINE/HOME,1,0,3,1,...,Admitted,Cornell,N,English,Male,White,Single,N,English,Male
3,1000634700270,15,0,0,0,ROUTINE/HOME,1,0,3,1,...,Admitted,LMH,N,English,Male,White,Single,N,English,Male
4,1001023900270,9,0,0,0,ROUTINE/HOME,1,0,2,1,...,Admitted,Cornell,N,ENG,Male,W,W,N,English,Male
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1699,999130900270,3,0,0,0,ROUTINE/HOME,1,0,2,2,...,Admitted,Cornell,HM,English,Female,Other Not Described,Single,H,English,Female
1700,999139900270,1,0,0,0,DISCHARGE HOME WITH HM HLTH SERV,0,0,3,2,...,Admitted,Cornell,U,English,Female,Declined,Widower,unknown,English,Female
1701,999619900270,11,0,0,0,DISCHARGE TO SNF,0,0,4,2,...,Admitted,LMH,N,Cantonese,Female,Asian,Single,N,Non-English,Female
1702,999912100270,11,0,0,0,DISCHARGE HOME WITH HM HLTH SERV,0,0,1,1,...,Admitted,Cornell,N,English,Male,Black/African Am.,Single,N,English,Male


In [255]:
descriptive_demo['race_c'] = descriptive_demo['RaceCode']

In [256]:
descriptive_demo['RaceCode'].value_counts()

White                   640
Black/African Am.       323
Other Not Described     225
Asian                   206
Declined                137
Unknown-Pt not Avail     64
W                        31
B                        22
O                        21
A                        16
Na Hawaii/Pac Island      5
D                         5
Chinese                   3
Asian Indian              2
Korean                    2
Other Pacif Islander      1
Black                     1
Name: RaceCode, dtype: int64

In [115]:
#descriptive_demo[descriptive_demo['clientvisitguid']==1103247800270]  ## Yiye's previously RaceCode was P

,clientvisitguid,LOS,30day,60day,90day,dischargedisposition,home,death,Elixhauser,Charlson,...,campus,ethnicity,LanguageCode,GenderCode,RaceCode,MaritalStatusCode,ethnicity_c,language_c,gender_c,race_c
345,1103247800270,9,0,0,0,ROUTINE/HOME,1,0,3,1,...,Cornell,H,Spanish,Female,Na Hawaii/Pac Island,Divorced,H,Non-English,Female,Na Hawaii/Pac Island


In [257]:
for i in range(len(descriptive_demo['clientvisitguid'])):   ## RaceCode is the column 20, race_c is the column 25
    if (descriptive_demo.iloc[i, 20] == 'W') or (descriptive_demo.iloc[i, 20] == 'White'):
        descriptive_demo.iloc[i, 25] = 'White'
    elif (descriptive_demo.iloc[i, 20] == 'A') or (descriptive_demo.iloc[i, 20] == 'Asian') or (descriptive_demo.iloc[i, 20] == 'Asian Indian') or (descriptive_demo.iloc[i, 20] == 'Chinese') or (descriptive_demo.iloc[i, 20] == 'Korean'):
        descriptive_demo.iloc[i, 25] = 'Asian' 
    elif (descriptive_demo.iloc[i, 20] == 'B') or (descriptive_demo.iloc[i, 20] == 'Black') or (descriptive_demo.iloc[i, 20] == 'Black/African Am.'):
        descriptive_demo.iloc[i, 25] = 'Black' 
    elif (descriptive_demo.iloc[i, 20] == 'D') or (descriptive_demo.iloc[i, 20] == 'Declined') or (descriptive_demo.iloc[i, 20] == 'Unknown-Pt not Avail'): 
        descriptive_demo.iloc[i, 25] = 'Unknown'
    else :
        descriptive_demo.iloc[i, 25] = 'Other'

In [258]:
descriptive_demo

,clientvisitguid,LOS,30day,60day,90day,dischargedisposition,home,death,Elixhauser,Charlson,...,campus,ethnicity,LanguageCode,GenderCode,RaceCode,MaritalStatusCode,ethnicity_c,language_c,gender_c,race_c
0,1000120800270,2,0,0,0,ROUTINE/HOME,1,0,4,4,...,LMH,U,English,Male,Other Not Described,Single,unknown,English,Male,Other
1,1000124200270,4,1,1,1,ROUTINE/HOME,1,0,4,2,...,LMH,N,English,Male,Black/African Am.,Single,N,English,Male,Black
2,1000155300270,15,0,0,0,ROUTINE/HOME,1,0,3,1,...,Cornell,N,English,Male,White,Single,N,English,Male,White
3,1000634700270,15,0,0,0,ROUTINE/HOME,1,0,3,1,...,LMH,N,English,Male,White,Single,N,English,Male,White
4,1001023900270,9,0,0,0,ROUTINE/HOME,1,0,2,1,...,Cornell,N,ENG,Male,W,W,N,English,Male,White
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1699,999130900270,3,0,0,0,ROUTINE/HOME,1,0,2,2,...,Cornell,HM,English,Female,Other Not Described,Single,H,English,Female,Other
1700,999139900270,1,0,0,0,DISCHARGE HOME WITH HM HLTH SERV,0,0,3,2,...,Cornell,U,English,Female,Declined,Widower,unknown,English,Female,Unknown
1701,999619900270,11,0,0,0,DISCHARGE TO SNF,0,0,4,2,...,LMH,N,Cantonese,Female,Asian,Single,N,Non-English,Female,Asian
1702,999912100270,11,0,0,0,DISCHARGE HOME WITH HM HLTH SERV,0,0,1,1,...,Cornell,N,English,Male,Black/African Am.,Single,N,English,Male,Black


In [259]:
descriptive_demo['marital_c'] = descriptive_demo['MaritalStatusCode']

In [260]:
descriptive_demo['MaritalStatusCode'].value_counts()

Married         633
Single          630
Widower         209
Divorced         97
M                45
S                36
Unknown          21
Separated        13
W                11
Life Partner      6
D                 2
X                 1
Name: MaritalStatusCode, dtype: int64

In [126]:
#descriptive_demo[descriptive_demo['MaritalStatusCode']=='M']  ## Yiye previously regard m as notmarried

,clientvisitguid,LOS,30day,60day,90day,dischargedisposition,home,death,Elixhauser,Charlson,...,ethnicity,LanguageCode,GenderCode,RaceCode,MaritalStatusCode,ethnicity_c,language_c,gender_c,race_c,marital_c
28,1008436800270,10,0,0,0,ROUTINE/HOME,1,0,1,1,...,H,SPA,Female,O,M,H,Non-English,Female,Other,M
29,1008487300270,5,0,0,0,ROUTINE/HOME,1,0,2,1,...,N,ENG,Male,W,M,N,English,Male,White,M
34,1011331400270,3,0,0,0,DISCHARGE HOME WITH HM HLTH SERV,0,0,2,2,...,N,ENG,Male,B,M,N,English,Male,Black,M
40,1013255800270,6,0,0,0,DISCHARGE HOME WITH HM HLTH SERV,0,0,3,2,...,N,ENG,Male,A,M,N,English,Male,Asian,M
73,1023407600270,2,1,1,1,DISCHARGE HOME WITH HM HLTH SERV,0,0,3,2,...,H,SPA,Female,O,M,H,Non-English,Female,Other,M
82,1026284100270,3,1,1,1,DISCHARGE HOME WITH HM HLTH SERV,0,0,1,1,...,H,SPA,Female,O,M,H,Non-English,Female,Other,M
102,1031544300270,2,0,0,0,DISCHARGE HOME WITH HM HLTH SERV,0,0,2,1,...,U,ENG,Female,W,M,unknown,English,Female,White,M
112,1033294400270,6,0,0,0,ROUTINE/HOME,1,0,2,1,...,N,ENG,Male,A,M,N,English,Male,Asian,M
116,1033542500270,7,0,1,1,TRANS/DISCHARGE TO/FM IP REHAB FAC,0,0,2,1,...,N,ENG,Male,A,M,N,English,Male,Asian,M
124,1035284600270,2,0,0,0,ROUTINE/HOME,1,0,2,2,...,N,CAN,Male,A,M,N,Non-English,Male,Asian,M


In [261]:
for i in range(len(descriptive_demo['MaritalStatusCode'])):   ## MaritalStatusCode is the column 21, marital_c is the column 26
    if (descriptive_demo.iloc[i, 21] == 'Married') or (descriptive_demo.iloc[i, 21] == 'M'):
        descriptive_demo.iloc[i, 26] = 'Married'
    else:
        descriptive_demo.iloc[i, 26] = 'NotMarried'  

In [262]:
descriptive_demo

,clientvisitguid,LOS,30day,60day,90day,dischargedisposition,home,death,Elixhauser,Charlson,...,ethnicity,LanguageCode,GenderCode,RaceCode,MaritalStatusCode,ethnicity_c,language_c,gender_c,race_c,marital_c
0,1000120800270,2,0,0,0,ROUTINE/HOME,1,0,4,4,...,U,English,Male,Other Not Described,Single,unknown,English,Male,Other,NotMarried
1,1000124200270,4,1,1,1,ROUTINE/HOME,1,0,4,2,...,N,English,Male,Black/African Am.,Single,N,English,Male,Black,NotMarried
2,1000155300270,15,0,0,0,ROUTINE/HOME,1,0,3,1,...,N,English,Male,White,Single,N,English,Male,White,NotMarried
3,1000634700270,15,0,0,0,ROUTINE/HOME,1,0,3,1,...,N,English,Male,White,Single,N,English,Male,White,NotMarried
4,1001023900270,9,0,0,0,ROUTINE/HOME,1,0,2,1,...,N,ENG,Male,W,W,N,English,Male,White,NotMarried
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1699,999130900270,3,0,0,0,ROUTINE/HOME,1,0,2,2,...,HM,English,Female,Other Not Described,Single,H,English,Female,Other,NotMarried
1700,999139900270,1,0,0,0,DISCHARGE HOME WITH HM HLTH SERV,0,0,3,2,...,U,English,Female,Declined,Widower,unknown,English,Female,Unknown,NotMarried
1701,999619900270,11,0,0,0,DISCHARGE TO SNF,0,0,4,2,...,N,Cantonese,Female,Asian,Single,N,Non-English,Female,Asian,NotMarried
1702,999912100270,11,0,0,0,DISCHARGE HOME WITH HM HLTH SERV,0,0,1,1,...,N,English,Male,Black/African Am.,Single,N,English,Male,Black,NotMarried


### Merge OSpercent

In [4]:
#order_EDDC = pd.read_csv(path + '2_order_EDDC_2012_2018.csv')
#order_taskocc= pd.read_csv(path + 'order_taskocc_2012_2018_new.csv')
order_notcancelled = pd.read_csv(path + 'order_EDDC_2012_2018_cv3_notcancelled.csv')
order_notcancelled.head()

,name,clientvisitguid,clientguid,createdwhen,ordersetname,enterrole,typecode,userguid,orderstatuscode,year
0,12 hour order check up to,9000834418200270,9000216018000200,2016-01-25 07:00:00,NaN,Nurse,Other,9000081299501190,COMP,2016
1,12 hour order check up to,9000948214000270,9000225902300200,2017-01-12 07:00:00,NaN,Nurse,Other,9000030646401190,COMP,2017
2,"Occult Blood, Stool",9000805374100270,9000071888900200,2015-11-02 14:00:00,NaN,NP,Diagnostic,9000002688401190,RESF,2015
3,Urine Culture,9000832536400270,9000143948600200,2016-01-18 09:00:00,NaN,Resident,Diagnostic,9000041766101190,RESF,2016
4,Liver Function Panel,9000975060500270,9000111865300200,2017-03-02 16:00:00,AM Lab Order Set,Resident,Diagnostic,9000118483601190,RESF,2017


In [5]:
len(order_notcancelled)

519576

In [6]:
order_notcancelled.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 519576 entries, 0 to 519575
Data columns (total 10 columns):
name               519576 non-null object
clientvisitguid    519576 non-null int64
clientguid         519576 non-null int64
createdwhen        519576 non-null object
ordersetname       255154 non-null object
enterrole          519576 non-null object
typecode           519576 non-null object
userguid           519576 non-null int64
orderstatuscode    519576 non-null object
year               519576 non-null int64
dtypes: int64(4), object(6)
memory usage: 39.6+ MB


In [276]:
#order_taskocc1 = order_taskocc[order_taskocc['TaskStatusCode']!='Canceled']

In [277]:
#len(order_taskocc1)

5586425

In [7]:
order_notcancelled['enterrole'].value_counts()

Resident               250831
PA                     139980
Nurse                   48635
Physician               45198
NP                      33357
Medical Resident          751
VerifyChemoTherapy        609
Attending Physician        94
Fellow                     94
Registered Nurse           27
Name: enterrole, dtype: int64

In [8]:
order_notcancelled2 = order_notcancelled[order_notcancelled['enterrole'].isin(['PA', 'Resident', 'Nurse', 'NP', 'Physician', 'VerifyChemoTherapy',
       'Fellow', 'Medical Resident', 'Attending Physician',
       'Registered Nurse', 'Nurse Anesthetist'])]

In [9]:
len(order_notcancelled2)

519576

In [ ]:
### OSpercent for EDDC order not cancelled

In [10]:
clientvisitguid_order = order_notcancelled2.groupby(['clientvisitguid'])[['name']].count()

In [11]:
clientvisitguid_os = order_notcancelled2.groupby(['clientvisitguid'])[['ordersetname']].apply(lambda x: x.notnull().sum())

In [12]:
print(len(clientvisitguid_order))
print(len(clientvisitguid_os))

2345
2345


In [13]:
clientvisitguid_order_os = pd.merge(left=clientvisitguid_order, right=clientvisitguid_os, on='clientvisitguid', how='left')

In [14]:
clientvisitguid_order_os

,name,ordersetname
clientvisitguid,,
9000556943700270,215,118
9000561002000270,252,107
9000562000000270,184,110
9000566520000270,341,145
9000567313100270,1022,487
...,...,...
9001284311200270,99,44
9001284903800270,85,49
9001284909400270,64,41


In [15]:
clientvisitguid_order_os['percent'] = clientvisitguid_order_os['ordersetname']/clientvisitguid_order_os['name']

In [16]:
clientvisitguid_order_os = clientvisitguid_order_os.reset_index()

In [17]:
clientvisitguid_order_os.columns

Index(['clientvisitguid', 'name', 'ordersetname', 'percent'], dtype='object')

In [18]:
clientvisitguid_order_os.to_excel(path + '2_OSpercent_EDDC_cv3_notcancelled.xlsx', index=None)

In [ ]:
### OSpercent for order not cancelled

In [305]:
clientvisitguid_order1 = order_taskocc1.groupby(['clientvisitguid'])[['name']].count()

In [306]:
clientvisitguid_os1 = order_taskocc1.groupby(['clientvisitguid'])[['ordersetname']].apply(lambda x: x.notnull().sum())

In [307]:
print(len(clientvisitguid_order1))
print(len(clientvisitguid_os1))

13687
13687


In [308]:
clientvisitguid_order_os1 = pd.merge(left=clientvisitguid_order1, right=clientvisitguid_os1, on='clientvisitguid', how='left')

In [309]:
clientvisitguid_order_os1

,name,ordersetname
clientvisitguid,,
279306000270,473,220
429822600270,14,8
503942200270,18455,1922
529725300270,4167,519
541079700270,937,207
...,...,...
1285095800270,37,18
1285155800270,359,38
1285281600270,62,37


In [310]:
clientvisitguid_order_os1['percent'] = clientvisitguid_order_os1['ordersetname']/clientvisitguid_order_os1['name']

In [311]:
clientvisitguid_order_os1 = clientvisitguid_order_os1.reset_index()

In [312]:
clientvisitguid_order_os1.columns

Index(['clientvisitguid', 'name', 'ordersetname', 'percent'], dtype='object')

In [313]:
clientvisitguid_order_os1.to_excel(path + '2_OSpercent.xlsx', index=None)

In [292]:
#check
order_taskocc2[order_taskocc2['clientvisitguid']==1285281600270]

,name,TaskStatusCode,clientvisitguid,clientguid,PerformedFromDtm,createdwhen,ordersetname,enterrole,typecode,userguid
88028,Basic Metabolic Panel,Performed,1285281600270,323865900200,2019-01-01 05:57:00.000,2018-12-31 17:42:22.040,MED - ED Admitted Patient Order Set (LMH),Physician,Diagnostic,999957774098810
101302,"Urinalysis, Dipstick with Microscopic Exam on ...",Performed,1285281600270,323865900200,2018-12-31 13:59:00.000,2018-12-31 13:58:46.993,ED Order Set - A General Comprehensive Order ...,Physician,Diagnostic,999969962298810
178474,Basic Metabolic Panel,Performed,1285281600270,323865900200,2018-12-31 13:59:00.000,2018-12-31 13:58:46.853,ED Order Set - A General Comprehensive Order ...,Physician,Diagnostic,999969962298810
265328,Diltiazem Inj,Performed,1285281600270,323865900200,2018-12-31 16:30:00.000,2018-12-31 16:30:32.730,NaN,Physician,Medication,999969962298810
361234,Lipid Profile for Westchester/LMH,Performed,1285281600270,323865900200,2019-01-01 05:57:00.000,2018-12-31 17:45:11.097,Lipid Profile AM Lab Order Set (LMH),Physician,Diagnostic,999957774098810
469186,Complete Blood Count with Differential,Performed,1285281600270,323865900200,2019-01-01 05:57:00.000,2018-12-31 17:42:22.023,MED - ED Admitted Patient Order Set (LMH),Physician,Diagnostic,999957774098810
487759,Normal Saline,Pending,1285281600270,323865900200,NaN,2018-12-31 13:58:47.160,NaN,Physician,Medication,999969962298810
588611,Liver Function Panel,Performed,1285281600270,323865900200,2019-01-01 05:57:00.000,2018-12-31 17:45:11.080,NaN,Physician,Diagnostic,999957774098810
1028628,ECG/12 Lead-POC ED,Performed,1285281600270,323865900200,2018-12-31 13:57:00.000,2018-12-31 13:58:47.023,ED Order Set - A General Comprehensive Order ...,Physician,Other,999969962298810
1066921,Heparin Inj,Performed,1285281600270,323865900200,2019-01-01 05:54:41.000,2018-12-31 17:42:21.517,Admit NYP Order Set..,Physician,Medication,999957774098810


In [193]:
check_order_eddc['createdwhen']

1398      2014-01-01 17:00:00
2505      2014-01-01 17:00:00
2506      2014-01-01 17:00:00
3019      2014-01-01 17:00:00
3020      2014-01-01 17:00:00
                 ...         
946934    2014-01-31 16:00:00
946935    2014-01-31 16:00:00
946936    2014-01-31 16:00:00
946937    2014-01-31 16:00:00
947879    2014-02-14 13:00:00
Name: createdwhen, Length: 1363, dtype: object

In [264]:
descriptive_ospercent = descriptive_demo.join(clientvisitguid_order_os, on='clientvisitguid', how='left')

In [265]:
descriptive_ospercent = descriptive_ospercent.drop(['name', 'ordersetname'], axis='columns')

In [266]:
descriptive_ospercent.columns

Index(['clientvisitguid', 'LOS', '30day', '60day', '90day',
       'dischargedisposition', 'home', 'death', 'Elixhauser', 'Charlson',
       'userQ', 'CCW', 'VTE', 'age', 'EDSite', 'FinalDispoCategory', 'campus',
       'ethnicity', 'LanguageCode', 'GenderCode', 'RaceCode',
       'MaritalStatusCode', 'ethnicity_c', 'language_c', 'gender_c', 'race_c',
       'marital_c', 'percent'],
      dtype='object')

In [267]:
descriptive_ospercent.columns = ['clientvisitguid', 'LOS', '30day', '60day', '90day', 
                       'dischargedisposition', 'home', 'death', 'Elixhauser', 'Charlson', 
                       'userQ', 'CCW', 'VTE', 'age', 'Edcampus', 'Eddecision', 
                       'campus', 'ethnicity_raw', 'language_raw', 'gender_raw', 'race_raw',
                       'marital_raw', 'ethnicity', 'language', 'gender', 'race', 'marital', 'OSpercent']   ## payor' , payorcode', not insurance and payor

In [268]:
#descriptive_ospercent.to_excel(path + '2_Descriptive_Table_lessfeature.xlsx', index = False)

In [269]:
descriptive_ospercent

,clientvisitguid,LOS,30day,60day,90day,dischargedisposition,home,death,Elixhauser,Charlson,...,language_raw,gender_raw,race_raw,marital_raw,ethnicity,language,gender,race,marital,OSpercent
0,1000120800270,2,0,0,0,ROUTINE/HOME,1,0,4,4,...,English,Male,Other Not Described,Single,unknown,English,Male,Other,NotMarried,0.490741
1,1000124200270,4,1,1,1,ROUTINE/HOME,1,0,4,2,...,English,Male,Black/African Am.,Single,N,English,Male,Black,NotMarried,0.304598
2,1000155300270,15,0,0,0,ROUTINE/HOME,1,0,3,1,...,English,Male,White,Single,N,English,Male,White,NotMarried,0.309249
3,1000634700270,15,0,0,0,ROUTINE/HOME,1,0,3,1,...,English,Male,White,Single,N,English,Male,White,NotMarried,0.332673
4,1001023900270,9,0,0,0,ROUTINE/HOME,1,0,2,1,...,ENG,Male,W,W,N,English,Male,White,NotMarried,0.353147
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1699,999130900270,3,0,0,0,ROUTINE/HOME,1,0,2,2,...,English,Female,Other Not Described,Single,H,English,Female,Other,NotMarried,0.245283
1700,999139900270,1,0,0,0,DISCHARGE HOME WITH HM HLTH SERV,0,0,3,2,...,English,Female,Declined,Widower,unknown,English,Female,Unknown,NotMarried,0.369863
1701,999619900270,11,0,0,0,DISCHARGE TO SNF,0,0,4,2,...,Cantonese,Female,Asian,Single,N,Non-English,Female,Asian,NotMarried,0.340858
1702,999912100270,11,0,0,0,DISCHARGE HOME WITH HM HLTH SERV,0,0,1,1,...,English,Male,Black/African Am.,Single,N,English,Male,Black,NotMarried,0.368750


### Merge insurance

In [270]:
insurance2 = pd.read_csv(path + 'insurance_2012_2018_pull2_new2.csv', header = 0)

In [271]:
insurance2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15999 entries, 0 to 15998
Data columns (total 2 columns):
clientvisitguid    15999 non-null int64
Financial Group    15999 non-null object
dtypes: int64(1), object(1)
memory usage: 250.1+ KB


In [297]:
descriptive_insurance = pd.merge(left = descriptive_ospercent, right = insurance2, 
                       left_on = 'clientvisitguid', right_on = 'clientvisitguid', how = 'left')
descriptive_insurance

,clientvisitguid,LOS,30day,60day,90day,dischargedisposition,home,death,Elixhauser,Charlson,...,gender_raw,race_raw,marital_raw,ethnicity,language,gender,race,marital,OSpercent,Financial Group
0,1000120800270,2,0,0,0,ROUTINE/HOME,1,0,4,4,...,Male,Other Not Described,Single,unknown,English,Male,Other,NotMarried,0.490741,MCDMGDCONTR
1,1000124200270,4,1,1,1,ROUTINE/HOME,1,0,4,2,...,Male,Black/African Am.,Single,N,English,Male,Black,NotMarried,0.304598,MCDMGDCONTR
2,1000155300270,15,0,0,0,ROUTINE/HOME,1,0,3,1,...,Male,White,Single,N,English,Male,White,NotMarried,0.309249,MCRFFS
3,1000155300270,15,0,0,0,ROUTINE/HOME,1,0,3,1,...,Male,White,Single,N,English,Male,White,NotMarried,0.309249,MCDMGDCONTR
4,1000634700270,15,0,0,0,ROUTINE/HOME,1,0,3,1,...,Male,White,Single,N,English,Male,White,NotMarried,0.332673,MCRFFS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2116,999130900270,3,0,0,0,ROUTINE/HOME,1,0,2,2,...,Female,Other Not Described,Single,H,English,Female,Other,NotMarried,0.245283,MCRFFS
2117,999139900270,1,0,0,0,DISCHARGE HOME WITH HM HLTH SERV,0,0,3,2,...,Female,Declined,Widower,unknown,English,Female,Unknown,NotMarried,0.369863,MCRFFS
2118,999619900270,11,0,0,0,DISCHARGE TO SNF,0,0,4,2,...,Female,Asian,Single,N,Non-English,Female,Asian,NotMarried,0.340858,MCRFFS
2119,999912100270,11,0,0,0,DISCHARGE HOME WITH HM HLTH SERV,0,0,1,1,...,Male,Black/African Am.,Single,N,English,Male,Black,NotMarried,0.368750,MCRFFS


In [298]:
## there're more rows
descriptive_insurance[descriptive_insurance.duplicated('clientvisitguid')==True]

,clientvisitguid,LOS,30day,60day,90day,dischargedisposition,home,death,Elixhauser,Charlson,...,gender_raw,race_raw,marital_raw,ethnicity,language,gender,race,marital,OSpercent,Financial Group
3,1000155300270,15,0,0,0,ROUTINE/HOME,1,0,3,1,...,Male,White,Single,N,English,Male,White,NotMarried,0.309249,MCDMGDCONTR
9,1002721000270,4,0,0,0,DISCHARGE HOME WITH HM HLTH SERV,0,0,3,1,...,Male,Asian,Single,N,Non-English,Male,Asian,NotMarried,0.265700,MCRMGDCONTR
16,1005144400270,2,0,0,0,ROUTINE/HOME,1,0,2,1,...,Male,O,S,N,Non-English,Male,Other,NotMarried,0.442105,SELFPAY
22,1005938300270,2,0,0,0,DISCHARGE TO ACUTE FACILITY,0,0,3,1,...,Male,Declined,Single,unknown,English,Male,Unknown,NotMarried,0.522727,MCDFFS
30,1008374100270,2,0,0,0,ROUTINE/HOME,1,0,1,1,...,Female,Black/African Am.,Single,N,English,Female,Black,NotMarried,0.434343,MCDFFS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2107,997602600270,4,0,0,0,DISCHARGE HOME WITH HM HLTH SERV,0,0,1,1,...,Male,White,Married,N,English,Male,White,Married,0.220779,BCIND
2110,997907700270,7,0,0,0,DISCHARGE HOME WITH HM HLTH SERV,0,0,1,1,...,Female,Other Not Described,Single,H,Non-English,Female,Other,NotMarried,0.395105,MCRFFS
2111,997907700270,7,0,0,0,DISCHARGE HOME WITH HM HLTH SERV,0,0,1,1,...,Female,Other Not Described,Single,H,Non-English,Female,Other,NotMarried,0.395105,MCRMGDNONCONTR
2114,998676900270,2,0,0,0,ROUTINE/HOME,1,0,1,1,...,Male,Black/African Am.,Separated,N,English,Male,Black,NotMarried,0.383562,MCDFFS


In [274]:
descriptive_insurance[descriptive_insurance['clientvisitguid'] == 1000155300270]

,clientvisitguid,LOS,30day,60day,90day,dischargedisposition,home,death,Elixhauser,Charlson,...,gender_raw,race_raw,marital_raw,ethnicity,language,gender,race,marital,OSpercent,Financial Group
2,1000155300270,15,0,0,0,ROUTINE/HOME,1,0,3,1,...,Male,White,Single,N,English,Male,White,NotMarried,0.309249,MCRFFS
3,1000155300270,15,0,0,0,ROUTINE/HOME,1,0,3,1,...,Male,White,Single,N,English,Male,White,NotMarried,0.309249,MCDMGDCONTR


In [275]:
descriptive_insurance['Financial Group'].value_counts()

MCRFFS            907
MCRMGDCONTR       431
MCDMGDCONTR       191
MCDFFS            152
COMMRMSDRG        144
BCPPO              75
COMMRCONTR         36
MCRMGDNONCONTR     36
MCDMGDNONCONTR     24
BCOTHER            21
BCIND              16
SELFPAY            14
COMMRNONCONTR      12
BCHMO               9
WORKCOMP            5
NOFAULT             1
RESEARCH            1
Name: Financial Group, dtype: int64

In [299]:
descriptive_insurance['payor'] = descriptive_insurance['Financial Group']

In [300]:
descriptive_insurance['payor'].isnull().sum()

46

In [278]:
descriptive_insurance

,clientvisitguid,LOS,30day,60day,90day,dischargedisposition,home,death,Elixhauser,Charlson,...,race_raw,marital_raw,ethnicity,language,gender,race,marital,OSpercent,Financial Group,payor
0,1000120800270,2,0,0,0,ROUTINE/HOME,1,0,4,4,...,Other Not Described,Single,unknown,English,Male,Other,NotMarried,0.490741,MCDMGDCONTR,MCDMGDCONTR
1,1000124200270,4,1,1,1,ROUTINE/HOME,1,0,4,2,...,Black/African Am.,Single,N,English,Male,Black,NotMarried,0.304598,MCDMGDCONTR,MCDMGDCONTR
2,1000155300270,15,0,0,0,ROUTINE/HOME,1,0,3,1,...,White,Single,N,English,Male,White,NotMarried,0.309249,MCRFFS,MCRFFS
3,1000155300270,15,0,0,0,ROUTINE/HOME,1,0,3,1,...,White,Single,N,English,Male,White,NotMarried,0.309249,MCDMGDCONTR,MCDMGDCONTR
4,1000634700270,15,0,0,0,ROUTINE/HOME,1,0,3,1,...,White,Single,N,English,Male,White,NotMarried,0.332673,MCRFFS,MCRFFS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2116,999130900270,3,0,0,0,ROUTINE/HOME,1,0,2,2,...,Other Not Described,Single,H,English,Female,Other,NotMarried,0.245283,MCRFFS,MCRFFS
2117,999139900270,1,0,0,0,DISCHARGE HOME WITH HM HLTH SERV,0,0,3,2,...,Declined,Widower,unknown,English,Female,Unknown,NotMarried,0.369863,MCRFFS,MCRFFS
2118,999619900270,11,0,0,0,DISCHARGE TO SNF,0,0,4,2,...,Asian,Single,N,Non-English,Female,Asian,NotMarried,0.340858,MCRFFS,MCRFFS
2119,999912100270,11,0,0,0,DISCHARGE HOME WITH HM HLTH SERV,0,0,1,1,...,Black/African Am.,Single,N,English,Male,Black,NotMarried,0.368750,MCRFFS,MCRFFS


In [301]:
for i in range(len(descriptive_insurance['clientvisitguid'])):   ## Financial Group is the column 28, payor is the column 29
    if (descriptive_insurance.iloc[i, 28] == 'MCRFFS') or (descriptive_insurance.iloc[i, 28] == 'MCRMGDCONTR') or (descriptive_insurance.iloc[i, 28] == 'MCRMGDNONCONTR'):
        descriptive_insurance.iloc[i, 29] = 'Medicare'
    elif (descriptive_insurance.iloc[i, 28] == 'MCDFFS') or (descriptive_insurance.iloc[i, 28] == 'MCDMGDCONTR') or (descriptive_insurance.iloc[i, 28] == 'MCDMGDNONCONTR'):
        descriptive_insurance.iloc[i, 29] = 'Medicaid' 
    elif (descriptive_insurance.iloc[i, 28] == 'BCOTHER') or (descriptive_insurance.iloc[i, 28] == 'BCIND') or (descriptive_insurance.iloc[i, 28] == 'COMMRNONCONTR') or (descriptive_insurance.iloc[i, 28] == 'BCHMO') or (descriptive_insurance.iloc[i, 28] == 'WORKCOMP') or (descriptive_insurance.iloc[i, 28] == 'NOFAULT') or (descriptive_insurance.iloc[i, 28] == 'RESEARCH') or (descriptive_insurance.iloc[i, 28] == 'COMMRMSDRG') or (descriptive_insurance.iloc[i, 28] == 'BCPPO') or (descriptive_insurance.iloc[i, 28] == 'COMMRCONTR'):
        descriptive_insurance.iloc[i, 29] = 'Commercial' 
    else :
        descriptive_insurance.iloc[i, 29] = 'Selfpay'

In [308]:
descriptive_insurance2 = descriptive_insurance.drop(['Financial Group'], axis='columns')

In [309]:
descriptive_insurance2

,clientvisitguid,LOS,30day,60day,90day,dischargedisposition,home,death,Elixhauser,Charlson,...,gender_raw,race_raw,marital_raw,ethnicity,language,gender,race,marital,OSpercent,payor
0,1000120800270,2,0,0,0,ROUTINE/HOME,1,0,4,4,...,Male,Other Not Described,Single,unknown,English,Male,Other,NotMarried,0.490741,Medicaid
1,1000124200270,4,1,1,1,ROUTINE/HOME,1,0,4,2,...,Male,Black/African Am.,Single,N,English,Male,Black,NotMarried,0.304598,Medicaid
2,1000155300270,15,0,0,0,ROUTINE/HOME,1,0,3,1,...,Male,White,Single,N,English,Male,White,NotMarried,0.309249,Medicare
3,1000155300270,15,0,0,0,ROUTINE/HOME,1,0,3,1,...,Male,White,Single,N,English,Male,White,NotMarried,0.309249,Medicaid
4,1000634700270,15,0,0,0,ROUTINE/HOME,1,0,3,1,...,Male,White,Single,N,English,Male,White,NotMarried,0.332673,Medicare
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2116,999130900270,3,0,0,0,ROUTINE/HOME,1,0,2,2,...,Female,Other Not Described,Single,H,English,Female,Other,NotMarried,0.245283,Medicare
2117,999139900270,1,0,0,0,DISCHARGE HOME WITH HM HLTH SERV,0,0,3,2,...,Female,Declined,Widower,unknown,English,Female,Unknown,NotMarried,0.369863,Medicare
2118,999619900270,11,0,0,0,DISCHARGE TO SNF,0,0,4,2,...,Female,Asian,Single,N,Non-English,Female,Asian,NotMarried,0.340858,Medicare
2119,999912100270,11,0,0,0,DISCHARGE HOME WITH HM HLTH SERV,0,0,1,1,...,Male,Black/African Am.,Single,N,English,Male,Black,NotMarried,0.368750,Medicare


In [310]:
descriptive_insurance2[descriptive_insurance2.duplicated(subset=['clientvisitguid', 'payor'])==True]

,clientvisitguid,LOS,30day,60day,90day,dischargedisposition,home,death,Elixhauser,Charlson,...,gender_raw,race_raw,marital_raw,ethnicity,language,gender,race,marital,OSpercent,payor
9,1002721000270,4,0,0,0,DISCHARGE HOME WITH HM HLTH SERV,0,0,3,1,...,Male,Asian,Single,N,Non-English,Male,Asian,NotMarried,0.265700,Medicare
22,1005938300270,2,0,0,0,DISCHARGE TO ACUTE FACILITY,0,0,3,1,...,Male,Declined,Single,unknown,English,Male,Unknown,NotMarried,0.522727,Medicaid
31,1008374100270,2,0,0,0,ROUTINE/HOME,1,0,1,1,...,Female,Black/African Am.,Single,N,English,Female,Black,NotMarried,0.434343,Medicaid
34,1008405700270,11,0,0,0,DISCHARGE TO SNF,0,0,4,3,...,Female,Black/African Am.,Single,N,English,Female,Black,NotMarried,0.346021,Medicare
41,1011065500270,5,0,0,0,ROUTINE/HOME,1,0,1,1,...,Male,Black/African Am.,Widower,N,English,Male,Black,NotMarried,0.318182,Medicaid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2102,997296900270,73,0,0,0,DISCHARGE TO SNF,0,0,7,2,...,Male,White,Married,H,Non-English,Male,White,Married,0.192506,Medicaid
2106,997602600270,4,0,0,0,DISCHARGE HOME WITH HM HLTH SERV,0,0,1,1,...,Male,White,Married,N,English,Male,White,Married,0.220779,Commercial
2107,997602600270,4,0,0,0,DISCHARGE HOME WITH HM HLTH SERV,0,0,1,1,...,Male,White,Married,N,English,Male,White,Married,0.220779,Commercial
2111,997907700270,7,0,0,0,DISCHARGE HOME WITH HM HLTH SERV,0,0,1,1,...,Female,Other Not Described,Single,H,Non-English,Female,Other,NotMarried,0.395105,Medicare


In [320]:
descriptive_insurance2[descriptive_insurance2['clientvisitguid']==997907700270]

,clientvisitguid,LOS,30day,60day,90day,dischargedisposition,home,death,Elixhauser,Charlson,...,gender_raw,race_raw,marital_raw,ethnicity,language,gender,race,marital,OSpercent,payor
2109,997907700270,7,0,0,0,DISCHARGE HOME WITH HM HLTH SERV,0,0,1,1,...,Female,Other Not Described,Single,H,Non-English,Female,Other,NotMarried,0.395105,Medicaid
2110,997907700270,7,0,0,0,DISCHARGE HOME WITH HM HLTH SERV,0,0,1,1,...,Female,Other Not Described,Single,H,Non-English,Female,Other,NotMarried,0.395105,Medicare
2111,997907700270,7,0,0,0,DISCHARGE HOME WITH HM HLTH SERV,0,0,1,1,...,Female,Other Not Described,Single,H,Non-English,Female,Other,NotMarried,0.395105,Medicare


In [305]:
## save to file
descriptive_insurance.columns

Index(['clientvisitguid', 'LOS', '30day', '60day', '90day',
       'dischargedisposition', 'home', 'death', 'Elixhauser', 'Charlson',
       'userQ', 'CCW', 'VTE', 'age', 'Edcampus', 'Eddecision', 'campus',
       'ethnicity_raw', 'language_raw', 'gender_raw', 'race_raw',
       'marital_raw', 'ethnicity', 'language', 'gender', 'race', 'marital',
       'OSpercent', 'Financial Group', 'payor'],
      dtype='object')

In [306]:
descriptive_insurance.to_csv(path + 'EDDC_order_2012_2018_cluster_MED_task_yifan.csv', index = False)
descriptive_insurance.to_excel(path + 'EDDC_order_2012_2018_cluster_MED_task_yifan.xlsx', index = False)

skip this part

In [223]:
#descriptive = pd.read_excel(path + 'Descriptive_Table_new.xlsx')

,clientvisitguid,LOS,30day,60day,90day,dischargedisposition,home,death,cluster_by30,Elixhauser,...,campus,ethnicity_raw,language_raw,gender,race_raw,marital,ethnicity,language,race,payor
0,1000120800270,2,0,0,0,ROUTINE/HOME,1,0,1,2,...,LMH,U,English,Male,Other Not Described,NotMarried,unknown,English,Other,Medicaid
1,1000155300270,15,0,0,0,ROUTINE/HOME,1,0,1,2,...,Cornell,N,English,Male,White,NotMarried,N,English,White,Medicare
2,1000155300270,15,0,0,0,ROUTINE/HOME,1,0,1,2,...,Cornell,N,English,Male,White,NotMarried,N,English,White,Medicaid
3,1000634700270,15,0,0,0,ROUTINE/HOME,1,0,2,3,...,LMH,N,English,Male,White,NotMarried,N,English,White,Medicare
4,1001176600270,1,0,0,0,DISCHARGE TO OTHER FAC NOT LISTED,0,0,1,2,...,Cornell,N,English,Male,White,NotMarried,N,English,White,Medicaid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1551,999130900270,3,0,0,0,ROUTINE/HOME,1,0,1,2,...,Cornell,HM,English,Female,Other Not Described,NotMarried,H,English,Other,Medicare
1552,999139900270,1,0,0,0,DISCHARGE HOME WITH HM HLTH SERV,0,0,1,2,...,Cornell,U,English,Female,Declined,NotMarried,unknown,English,Unknown,Medicare
1553,999619900270,11,0,0,0,DISCHARGE TO SNF,0,0,2,2,...,LMH,N,Cantonese,Female,Asian,NotMarried,N,Non-English,Asian,Medicare
1554,999912100270,11,0,0,0,DISCHARGE HOME WITH HM HLTH SERV,0,0,1,1,...,Cornell,N,English,Male,Black/African Am.,NotMarried,N,English,Black,Medicare


In [ ]:
# excel check repeated payor, and change payor name accordingly
 ## Medicare + Medicaid  = Dual
 ## Medicare + Commercial = Medicare
 ## Medicaid + Commercial = Medicaid
 ## Selfpay + Commercial = Commercial
 ## Medicare + Selfpay + Commercial = Medicare

In [85]:
duplicated_medicare = list(descriptive[(descriptive.duplicated(['clientvisitguid'])==True) & (descriptive['payor']=='Medicare')]['clientvisitguid'])

In [86]:
medicare_medicaid = []
medicare_other = []
for i in duplicated_medicare:
    if descriptive[descriptive['clientvisitguid']==i]['payor'].any()=='Medicaid':
        medicare_medicaid.append(i)
    elif (descriptive[descriptive['clientvisitguid']==i]['payor'].any()=='Commercial') or (descriptive[descriptive['clientvisitguid']==i]['payor'].any()=='Selfpay'):
        medicare_other.append(i)

In [229]:
#for i in medicare_medicaid:
#    for j in medicare_other:
#        if i == j:
#            print(i)

In [87]:
for i in medicare_medicaid:
    descriptive.loc[descriptive['clientvisitguid']==i, 'payor']='Dual'
    descriptive.drop_duplicates(subset = ['clientvisitguid', 'payor'], keep = 'first', inplace = True)

In [88]:
for i in medicare_other:
    descriptive.loc[descriptive['clientvisitguid']==i, 'payor']='Medicare'
    descriptive.drop_duplicates(subset = ['clientvisitguid', 'payor'], keep = 'first', inplace = True)

In [95]:
descriptive.to_excel(path + 'table.xlsx', index= False)

In [198]:
#duplicated_medicaid = list(descriptive[(descriptive.duplicated(['clientvisitguid'])==True) & (descriptive['payor']=='Medicaid')]['clientvisitguid'])

In [240]:
#medicaid_other = []
#for i in duplicated_medicaid:
#    if (descriptive[descriptive['clientvisitguid']==i]['payor'].any()=='Commercial') or (descriptive[descriptive['clientvisitguid']==i]['payor'].any()=='Selfpay'):
#        medicaid_other.append(i)
#for i in medicaid_other:
#    descriptive.loc[descriptive['clientvisitguid']==i, 'payor']='Medicaid'
#    descriptive.drop_duplicates(subset = ['clientvisitguid', 'payor'], keep = 'first', inplace = True)

In [199]:
#duplicated_commercial = list(descriptive[(descriptive.duplicated(['clientvisitguid'])==True) & (descriptive['payor']=='Commercial')]['clientvisitguid'])

In [245]:
#commercial_selfpay = []
#for i in duplicated_commercial:
#    if (descriptive[descriptive['clientvisitguid']==i]['payor'].any()=='Selfpay'):
#        commercial_selfpay.append(i)
#for i in commercial_selfpay:
#    descriptive.loc[descriptive['clientvisitguid']==i, 'payor']='Commercial'
#    descriptive.drop_duplicates(subset = ['clientvisitguid', 'payor'], keep = 'first', inplace = True)

In [256]:
descriptive[descriptive.duplicated(['clientvisitguid'])==True]

,clientvisitguid,LOS,30day,60day,90day,dischargedisposition,home,death,cluster_by30,Elixhauser,...,campus,ethnicity_raw,language_raw,gender,race_raw,marital,ethnicity,language,race,payor
2,1000155300270,15,0,0,0,ROUTINE/HOME,1,0,1,2,...,Cornell,N,English,Male,White,NotMarried,N,English,White,Medicaid
18,1008374100270,2,0,0,0,ROUTINE/HOME,1,0,1,1,...,Cornell,N,English,Female,Black/African Am.,NotMarried,N,English,Black,Medicaid
34,1013255800270,6,0,0,0,DISCHARGE HOME WITH HM HLTH SERV,0,0,1,2,...,Cornell,N,ENG,Male,A,Married,N,English,Asian,Commercial
39,1014970200270,8,0,0,0,ROUTINE/HOME,1,0,2,1,...,Cornell,N,English,Male,White,NotMarried,N,English,White,Medicaid
45,1017710500270,1,0,0,0,DISCHARGE HOME WITH HM HLTH SERV,0,0,1,2,...,Cornell,N,English,Male,Asian,Married,N,English,Asian,Medicaid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1481,980304600270,6,0,0,0,DISCHARGE HOME WITH HM HLTH SERV,0,0,2,2,...,LMH,N,English,Female,White,NotMarried,N,English,White,Commercial
1486,981677900270,11,0,0,0,DISCHARGE TO SNF,0,0,1,3,...,LMH,N,Mandarin,Female,Asian,Married,N,Non-English,Asian,Medicaid
1491,983404800270,0,0,0,0,ROUTINE/HOME,1,0,1,1,...,Cornell,N,English,Female,White,NotMarried,N,English,White,Medicaid
1534,994909800270,10,0,1,1,DISCHARGE HOME WITH HM HLTH SERV,0,0,1,3,...,LMH,HM,Spanish,Female,Other Not Described,NotMarried,H,Non-English,Other,Commercial


## Merge elixhauser10

In [234]:
table = pd.read_excel(path + 'EDDC_order_2012_2018_cluster_MED_task_yifanfinal_1651.xlsx')

In [235]:
table

,clientvisitguid,LOS,30day,60day,90day,dischargedisposition,home,death,Elixhauser,Charlson,...,gender_raw,race_raw,marital_raw,ethnicity,language,gender,race,marital,OSpercent_EDDC,payor
0,1000120800270,2,0,0,0,ROUTINE/HOME,1,0,4,4,...,Male,Other Not Described,Single,unknown,English,Male,Other,NotMarried,0.490741,Medicaid
1,1000124200270,4,1,1,1,ROUTINE/HOME,1,0,4,2,...,Male,Black/African Am.,Single,N,English,Male,Black,NotMarried,0.304598,Medicaid
2,1000155300270,15,0,0,0,ROUTINE/HOME,1,0,3,1,...,Male,White,Single,N,English,Male,White,NotMarried,0.309249,Dual
3,1000634700270,15,0,0,0,ROUTINE/HOME,1,0,3,1,...,Male,White,Single,N,English,Male,White,NotMarried,0.332673,Medicare
4,1001023900270,9,0,0,0,ROUTINE/HOME,1,0,2,1,...,Male,W,W,N,English,Male,White,NotMarried,0.353147,Medicare
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1646,999130900270,3,0,0,0,ROUTINE/HOME,1,0,2,2,...,Female,Other Not Described,Single,H,English,Female,Other,NotMarried,0.245283,Medicare
1647,999139900270,1,0,0,0,DISCHARGE HOME WITH HM HLTH SERV,0,0,3,2,...,Female,Declined,Widower,unknown,English,Female,Unknown,NotMarried,0.369863,Medicare
1648,999619900270,11,0,0,0,DISCHARGE TO SNF,0,0,4,2,...,Female,Asian,Single,N,Non-English,Female,Asian,NotMarried,0.340858,Medicare
1649,999912100270,11,0,0,0,DISCHARGE HOME WITH HM HLTH SERV,0,0,1,1,...,Male,Black/African Am.,Single,N,English,Male,Black,NotMarried,0.368750,Medicare


In [236]:
elixhauser10 = pd.read_excel(path + 'elixhauser10.xlsx')

In [237]:
elixhauser10

,ID,chf,carit,valv,pcd,pvd,hypunc,hypc,para,ond,...,alcohol,drug,psycho,depre,score,index,wscore_ahrq,wscore_vw,windex_ahrq,windex_vw
0,556943700270,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1-4,9,7,>=5,>=5
1,561002000270,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1-4,11,5,>=5,>=5
2,562000000270,1,1,0,0,0,0,0,0,0,...,0,0,0,0,3,1-4,16,16,>=5,>=5
3,566520000270,1,0,0,0,0,0,0,0,0,...,0,0,0,0,2,1-4,20,12,>=5,>=5
4,567313100270,1,1,1,0,0,0,0,0,0,...,1,0,0,1,5,>=5,3,8,1-4,>=5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2286,1284013900270,1,0,0,0,0,0,0,0,0,...,0,0,0,0,3,1-4,12,10,>=5,>=5
2287,1284311200270,1,1,0,0,0,0,0,0,0,...,0,0,0,0,2,1-4,9,12,>=5,>=5
2288,1284903800270,1,1,0,0,0,0,0,0,0,...,0,0,0,0,3,1-4,20,17,>=5,>=5
2289,1284909400270,1,1,0,0,0,0,0,0,0,...,0,0,0,0,2,1-4,9,12,>=5,>=5


In [238]:
table_elixhauser10 = pd.merge(left = table, right = elixhauser10, 
                              left_on = 'clientvisitguid', right_on = 'ID', how='left')

In [239]:
table_elixhauser10

,clientvisitguid,LOS,30day,60day,90day,dischargedisposition,home,death,Elixhauser,Charlson,...,alcohol,drug,psycho,depre,score,index,wscore_ahrq,wscore_vw,windex_ahrq,windex_vw
0,1000120800270,2,0,0,0,ROUTINE/HOME,1,0,4,4,...,0,0,0,0,4,1-4,18,15,>=5,>=5
1,1000124200270,4,1,1,1,ROUTINE/HOME,1,0,4,2,...,0,0,0,0,4,1-4,14,17,>=5,>=5
2,1000155300270,15,0,0,0,ROUTINE/HOME,1,0,3,1,...,0,0,0,0,3,1-4,15,16,>=5,>=5
3,1000634700270,15,0,0,0,ROUTINE/HOME,1,0,3,1,...,0,0,0,0,3,1-4,9,11,>=5,>=5
4,1001023900270,9,0,0,0,ROUTINE/HOME,1,0,2,1,...,0,0,0,0,2,1-4,9,12,>=5,>=5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1646,999130900270,3,0,0,0,ROUTINE/HOME,1,0,2,2,...,0,0,0,0,2,1-4,9,7,>=5,>=5
1647,999139900270,1,0,0,0,DISCHARGE HOME WITH HM HLTH SERV,0,0,3,2,...,0,0,0,0,3,1-4,14,12,>=5,>=5
1648,999619900270,11,0,0,0,DISCHARGE TO SNF,0,0,4,2,...,0,0,0,0,4,1-4,34,18,>=5,>=5
1649,999912100270,11,0,0,0,DISCHARGE HOME WITH HM HLTH SERV,0,0,1,1,...,0,0,0,0,1,1-4,9,7,>=5,>=5


## Merge ordercountby_id_generic_taskstatus

In [314]:
table2 = pd.read_excel(path + 'EDDC_order_2012_2018_cluster_MED_task_opioid_1651_yifan.xlsx')

In [315]:
table2

,ylclientvisitguid,clientvisitguid_x,EDDC_all,EDDC_notcancelled_noV0,EDDC_notcancelled,Hfcluster_allorders,Hfcluster_notcancelled_noV0,Hfcluster_notcancelled,HFDRG,LOS,...,alcohol,drug,psycho,depre,score,index,wscore_ahrq,wscore_vw,windex_ahrq,windex_vw
0,1000120800270,9001000120800270,1,1,1,2,2,1,1,2,...,0,0,0,0,4,1-4,18,15,>=5,>=5
1,1000124200270,9001000124200270,0,0,1,2,2,1,1,4,...,0,0,0,0,4,1-4,14,17,>=5,>=5
2,1000155300270,9001000155300270,1,1,1,2,2,1,1,15,...,0,0,0,0,3,1-4,15,16,>=5,>=5
3,1000634700270,9001000634700270,1,1,1,0,0,0,0,15,...,0,0,0,0,3,1-4,9,11,>=5,>=5
4,1001023900270,9001001023900270,0,0,1,2,2,1,1,9,...,0,0,0,0,2,1-4,9,12,>=5,>=5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1646,999130900270,9000999130900270,1,1,1,2,2,1,1,3,...,0,0,0,0,2,1-4,9,7,>=5,>=5
1647,999139900270,9000999139900270,1,1,1,2,2,1,1,1,...,0,0,0,0,3,1-4,14,12,>=5,>=5
1648,999619900270,9000999619900270,1,1,1,0,0,0,0,11,...,0,0,0,0,4,1-4,34,18,>=5,>=5
1649,999912100270,9000999912100270,1,1,1,2,2,1,1,11,...,0,0,0,0,1,1-4,9,7,>=5,>=5


In [316]:
ordercountby_id_generic_taskstatus = pd.read_excel('/Users/fan/Desktop/Fullhouse/WCM/Opioid/ordercountby_id_generic_taskstatus_dropduplicates_filtered_0305.xlsx')

In [317]:
ordercountby_id_generic_taskstatus

,clientvisitguid,Generic,Not Performed,Overdue,Pending,Performed
0,279306000270,Fentanyl,NaN,NaN,2.0,NaN
1,503942200270,Fentanyl,2.0,NaN,12.0,123.0
2,503942200270,Hydromorphone,7.0,NaN,25.0,823.0
3,503942200270,Meperidine,NaN,NaN,NaN,1.0
4,503942200270,Morphine,NaN,NaN,NaN,3.0
...,...,...,...,...,...,...
12758,1284379300270,Fentanyl,1.0,NaN,NaN,10.0
12759,1284754600270,Oxycodone,NaN,NaN,1.0,NaN
12760,1284928000270,Hydromorphone,NaN,NaN,2.0,18.0
12761,1284928000270,Meperidine,NaN,NaN,NaN,1.0


In [318]:
ordercountby_id_generic_taskstatus = ordercountby_id_generic_taskstatus.fillna(0)

In [319]:
ordercountby_id_generic_taskstatus[ordercountby_id_generic_taskstatus['clientvisitguid']==1008380900270]

,clientvisitguid,Generic,Not Performed,Overdue,Pending,Performed
8090,1008380900270,Morphine,0.0,0.0,0.0,19.0
8091,1008380900270,Oxycodone,1.0,0.0,1.0,29.0


In [320]:
ordercountby_id_generic_taskstatus['Generic'].unique()

array(['Fentanyl', 'Hydromorphone', 'Meperidine', 'Morphine', 'Oxycodone',
       'Methadone', 'Codeine', 'Hydrocodone', 'Diphenoxylate'],
      dtype=object)

In [322]:
ordercountby_id_generic_taskstatus.columns

Index(['clientvisitguid', 'Generic', 'Not Performed', 'Overdue', 'Pending',
       'Performed'],
      dtype='object')

In [323]:
opioid = ordercountby_id_generic_taskstatus.pivot_table(index='clientvisitguid', 
                                               columns='Generic', 
                                               values=['Not Performed', 'Overdue', 'Pending', 'Performed'])

In [328]:
opioid  ## 6820 clientvisitguids

Not Performed                                     \
Generic               Codeine Diphenoxylate Fentanyl Hydrocodone   
clientvisitguid                                                    
279306000270              NaN           NaN      0.0         NaN   
503942200270              NaN           NaN      2.0         NaN   
529725300270              NaN           NaN      2.0         NaN   
541079700270              NaN           NaN      0.0         NaN   
544801700270              NaN           NaN      0.0         NaN   
...                       ...           ...      ...         ...   
1284299400270             NaN           NaN      NaN         NaN   
1284379300270             NaN           NaN      1.0         NaN   
1284754600270             NaN           NaN      NaN         NaN   
1284928000270             NaN           NaN      NaN         NaN   
1284967200270             NaN           NaN      0.0         NaN   

                                                                      Overdue  \
Generic         Hydromorphone Meperidine Methadone Morphine Oxycodone Codeine   
clientvisitguid                                                                 
279306000270              NaN        NaN       NaN      NaN       NaN     NaN   
503942200270              7.0        0.0       NaN      0.0       1.0     NaN   
529725300270              2.0        NaN       NaN      1.0       4.0     NaN   
541079700270              NaN        NaN       NaN      0.0       0.0     NaN   
544801700270              0.0        NaN       NaN      NaN       0.0     NaN   
...                       ...        ...       ...      ...       ...     ...   
1284299400270             NaN        NaN       NaN      0.0       NaN     NaN   
1284379300270             NaN        NaN       NaN      NaN       NaN     NaN   
1284754600270             NaN        NaN       NaN      NaN       0.0     NaN   
1284928000270             0.0        0.0       NaN      NaN       NaN     NaN   
1284967200270             NaN        NaN       NaN      NaN       NaN     NaN   

                 ...   Pending Performed                                     \
Generic          ... Oxycodone   Codeine Diphenoxylate Fentanyl Hydrocodone   
clientvisitguid  ...                                                          
279306000270     ...       NaN       NaN           NaN      0.0         NaN   
503942200270     ...       4.0       NaN           NaN    123.0         NaN   
529725300270     ...       6.0       NaN           NaN     13.0         NaN   
541079700270     ...       1.0       NaN           NaN      1.0         NaN   
544801700270     ...       1.0       NaN           NaN      1.0         NaN   
...              ...       ...       ...           ...      ...         ...   
1284299400270    ...       NaN       NaN           NaN      NaN         NaN   
1284379300270    ...       NaN       NaN           NaN     10.0         NaN   
1284754600270    ...       1.0       NaN           NaN      NaN         NaN   
1284928000270    ...       NaN       NaN           NaN      NaN         NaN   
1284967200270    ...       NaN       NaN           NaN      4.0         NaN   

                                                                       
Generic         Hydromorphone Meperidine Methadone Morphine Oxycodone  
clientvisitguid                                                        
279306000270              NaN        NaN       NaN      NaN       NaN  
503942200270            823.0        1.0       NaN      3.0     231.0  
529725300270            134.0        NaN       NaN     17.0     116.0  
541079700270              NaN        NaN       NaN      1.0       8.0  
544801700270              1.0        NaN       NaN      NaN      11.0  
...                       ...        ...       ...      ...       ...  
1284299400270             NaN        NaN       NaN      1.0       NaN  
1284379300270             NaN        NaN       NaN      NaN       NaN  
1284754600270             NaN

In [325]:
opioid_filled = opioid.fillna(0)

In [327]:
opioid_filled = opioid_filled.reset_index()

In [329]:
opioid_filled

clientvisitguid Not Performed                                     \
Generic                       Codeine Diphenoxylate Fentanyl Hydrocodone   
0          279306000270           0.0           0.0      0.0         0.0   
1          503942200270           0.0           0.0      2.0         0.0   
2          529725300270           0.0           0.0      2.0         0.0   
3          541079700270           0.0           0.0      0.0         0.0   
4          544801700270           0.0           0.0      0.0         0.0   
...                 ...           ...           ...      ...         ...   
6815      1284299400270           0.0           0.0      0.0         0.0   
6816      1284379300270           0.0           0.0      1.0         0.0   
6817      1284754600270           0.0           0.0      0.0         0.0   
6818      1284928000270           0.0           0.0      0.0         0.0   
6819      1284967200270           0.0           0.0      0.0         0.0   

                                                               ...   Pending  \
Generic Hydromorphone Meperidine Methadone Morphine Oxycodone  ... Oxycodone   
0                 0.0        0.0       0.0      0.0       0.0  ...       0.0   
1                 7.0        0.0       0.0      0.0       1.0  ...       4.0   
2                 2.0        0.0       0.0      1.0       4.0  ...       6.0   
3                 0.0        0.0       0.0      0.0       0.0  ...       1.0   
4                 0.0        0.0       0.0      0.0       0.0  ...       1.0   
...               ...        ...       ...      ...       ...  ...       ...   
6815              0.0        0.0       0.0      0.0       0.0  ...       0.0   
6816              0.0        0.0       0.0      0.0       0.0  ...       0.0   
6817              0.0        0.0       0.0      0.0       0.0  ...       1.0   
6818              0.0        0.0       0.0      0.0       0.0  ...       0.0   
6819              0.0        0.0       0.0      0.0       0.0  ...       0.0   

        Performed                                                              \
Generic   Codeine Diphenoxylate Fentanyl Hydrocodone Hydromorphone Meperidine   
0             0.0           0.0      0.0         0.0           0.0        0.0   
1             0.0           0.0    123.0         0.0         823.0        1.0   
2             0.0           0.0     13.0         0.0         134.0        0.0   
3             0.0           0.0      1.0         0.0           0.0        0.0   
4             0.0           0.0      1.0         0.0           1.0        0.0   
...           ...           ...      ...         ...           ...        ...   
6815          0.0           0.0      0.0         0.0           0.0        0.0   
6816          0.0           0.0     10.0         0.0           0.0        0.0   
6817          0.0           0.0      0.0         0.0           0.0        0.0   
6818          0.0           0.0      0.0         0.0          18.0        1.0   
6819          0.0           0.0      4.0         0.0           0.0        0.0   

                                      
Generic Methadone Morphine Oxycodone  
0             0.0      0.0       0.0  
1             0.0      3.0     231.0  
2             0.0     17.0     116.0  
3             0.0      1.0       8.0  
4             0.0      0.0      11.0  
...           ...      ...       ...  
6815          0.0      1.0       0.0  
6816          0.0      0.0       0.0  
6817          0.0      0.0       0.0  
6818          0.0      0.0       0.0  
6819          0.0      0.0       0.0  

[6820 rows x 37 columns]

In [330]:
table_opioid = pd.merge(left = table2, right = opioid_filled, 
                        left_on = 'ylclientvisitguid', right_on = 'clientvisitguid', how='left')

In [331]:
table_opioid

,ylclientvisitguid,clientvisitguid_x,EDDC_all,EDDC_notcancelled_noV0,EDDC_notcancelled,Hfcluster_allorders,Hfcluster_notcancelled_noV0,Hfcluster_notcancelled,HFDRG,LOS,...,"(Pending, Oxycodone)","(Performed, Codeine)","(Performed, Diphenoxylate)","(Performed, Fentanyl)","(Performed, Hydrocodone)","(Performed, Hydromorphone)","(Performed, Meperidine)","(Performed, Methadone)","(Performed, Morphine)","(Performed, Oxycodone)"
0,1000120800270,9001000120800270,1,1,1,2,2,1,1,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1000124200270,9001000124200270,0,0,1,2,2,1,1,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1000155300270,9001000155300270,1,1,1,2,2,1,1,15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1000634700270,9001000634700270,1,1,1,0,0,0,0,15,...,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1001023900270,9001001023900270,0,0,1,2,2,1,1,9,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1646,999130900270,9000999130900270,1,1,1,2,2,1,1,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1647,999139900270,9000999139900270,1,1,1,2,2,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1648,999619900270,9000999619900270,1,1,1,0,0,0,0,11,...,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0
1649,999912100270,9000999912100270,1,1,1,2,2,1,1,11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [333]:
table_opioid[table_opioid['ylclientvisitguid']==1008380900270]

,ylclientvisitguid,clientvisitguid_x,EDDC_all,EDDC_notcancelled_noV0,EDDC_notcancelled,Hfcluster_allorders,Hfcluster_notcancelled_noV0,Hfcluster_notcancelled,HFDRG,LOS,...,"(Pending, Oxycodone)","(Performed, Codeine)","(Performed, Diphenoxylate)","(Performed, Fentanyl)","(Performed, Hydrocodone)","(Performed, Hydromorphone)","(Performed, Meperidine)","(Performed, Methadone)","(Performed, Morphine)","(Performed, Oxycodone)"
26,1008380900270,9001008380900270,1,1,1,2,2,1,1,7,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.0,29.0


In [334]:
table_opioid.columns

Index([           'ylclientvisitguid',            'clientvisitguid_x',
                           'EDDC_all',       'EDDC_notcancelled_noV0',
                  'EDDC_notcancelled',          'Hfcluster_allorders',
        'Hfcluster_notcancelled_noV0',       'Hfcluster_notcancelled',
                              'HFDRG',                          'LOS',
       ...
             ('Pending', 'Oxycodone'),       ('Performed', 'Codeine'),
       ('Performed', 'Diphenoxylate'),      ('Performed', 'Fentanyl'),
         ('Performed', 'Hydrocodone'), ('Performed', 'Hydromorphone'),
          ('Performed', 'Meperidine'),     ('Performed', 'Methadone'),
            ('Performed', 'Morphine'),     ('Performed', 'Oxycodone')],
      dtype='object', length=113)

In [335]:
table_opioid.loc[:, ('Not Performed', 'Codeine'):] = table_opioid.loc[:, ('Not Performed', 'Codeine'):].fillna(0)

In [336]:
table_opioid

,ylclientvisitguid,clientvisitguid_x,EDDC_all,EDDC_notcancelled_noV0,EDDC_notcancelled,Hfcluster_allorders,Hfcluster_notcancelled_noV0,Hfcluster_notcancelled,HFDRG,LOS,...,"(Pending, Oxycodone)","(Performed, Codeine)","(Performed, Diphenoxylate)","(Performed, Fentanyl)","(Performed, Hydrocodone)","(Performed, Hydromorphone)","(Performed, Meperidine)","(Performed, Methadone)","(Performed, Morphine)","(Performed, Oxycodone)"
0,1000120800270,9001000120800270,1,1,1,2,2,1,1,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1000124200270,9001000124200270,0,0,1,2,2,1,1,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1000155300270,9001000155300270,1,1,1,2,2,1,1,15,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1000634700270,9001000634700270,1,1,1,0,0,0,0,15,...,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1001023900270,9001001023900270,0,0,1,2,2,1,1,9,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1646,999130900270,9000999130900270,1,1,1,2,2,1,1,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1647,999139900270,9000999139900270,1,1,1,2,2,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1648,999619900270,9000999619900270,1,1,1,0,0,0,0,11,...,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0
1649,999912100270,9000999912100270,1,1,1,2,2,1,1,11,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [337]:
table_opioid.to_excel(path + 'EDDC_order_2012_2018_cluster_MED_task_opioid_1651_yifan.xlsx', index=None)

For each clientvisitguid, for each generic, how many performed generic orders were ordered from order set

In [247]:
table_opioid = pd.read_excel(path + 'EDDC_order_2012_2018_cluster_MED_task_opioid_yifanfinal_1651.xlsx', index=None)

In [248]:
table_opioid[table_opioid['clientvisitguid']==1002827500270]

,clientvisitguid,LOS,30day,60day,90day,dischargedisposition,home,death,Elixhauser,Charlson,...,"Overdue, Morphine","Pending, Morphine","Performed, Morphine","Rescheduled, Morphine","Canceled, Oxycodone","Not Performed, Oxycodone","Overdue, Oxycodone","Pending, Oxycodone","Performed, Oxycodone","Rescheduled, Oxycodone"
8,1002827500270,26,0,0,0,ROUTINE/HOME,1,0,3,1,...,0,0,0,0,2,0,0,0,23,0


In [249]:
table_opioidonly_performed = table_opioid.loc[:,['clientvisitguid','Performed, Codeine',
                    'Performed, Diphenoxylate', 'Performed, Fentanyl', 'Performed, Hydrocodone', 
                    'Performed, Hydromorphone', 'Performed, Meperidine', 'Performed, Methadone', 
                    'Performed, Morphine', 'Performed, Oxycodone']]

In [250]:
table_opioidonly_performed.describe()

,clientvisitguid,"Performed, Codeine","Performed, Diphenoxylate","Performed, Fentanyl","Performed, Hydrocodone","Performed, Hydromorphone","Performed, Meperidine","Performed, Methadone","Performed, Morphine","Performed, Oxycodone"
count,1.651000e+03,1651.000000,1651.000000,1651.000000,1651.000000,1651.000000,1651.000000,1651.000000,1651.000000,1651.000000
mean,1.029461e+12,0.119322,0.003028,0.893398,0.036947,2.479104,0.008480,0.274379,0.570563,2.400363
std,1.462306e+11,1.940243,0.123054,3.498159,0.591222,18.764667,0.139004,2.956729,2.593293,14.753702
min,5.569437e+11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,9.134060e+11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.029597e+12,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.151775e+12,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.285017e+12,50.000000,5.000000,60.000000,14.000000,444.000000,4.000000,69.000000,37.000000,502.000000


In [251]:
table_opioidonly_performed.isna().sum()

clientvisitguid             0
Performed, Codeine          0
Performed, Diphenoxylate    0
Performed, Fentanyl         0
Performed, Hydrocodone      0
Performed, Hydromorphone    0
Performed, Meperidine       0
Performed, Methadone        0
Performed, Morphine         0
Performed, Oxycodone        0
dtype: int64

In [253]:
medication_opioid = pd.read_csv('/Users/fan/Desktop/Fullhouse/WCM/Opioid/medication_opioid_12feature_dropduplicates_filtered.csv')

In [254]:
medication_opioid

,name,TaskStatusCode,clientvisitguid,clientguid,PerformedFromDtm,createdwhen,ordersetname,enterrole,typecode,userguid,Generic,OrderName
0,(Omnicell Override No Task),Not Performed,1167028600270,227099700200,NaN,2015-06-02 08:46:22.450,NaN,Nurse,Medication,999970898698810,NaN,NaN
1,(Omnicell Override No Task),Not Performed,1383411900270,220272700200,NaN,2016-09-20 08:02:44.663,NaN,Nurse,Medication,999970898698810,NaN,NaN
2,(Omnicell Override No Task),Not Performed,1389934800270,219103000200,NaN,2016-09-22 13:24:47.310,NaN,Nurse,Medication,999970898698810,NaN,NaN
3,(Omnicell Override No Task),Not Performed,1394688500270,254335500200,NaN,2016-09-13 16:16:59.980,NaN,Nurse,Medication,999970898698810,NaN,NaN
4,(Omnicell Override No Task),Not Performed,1432129500270,82049600200,NaN,2016-12-07 07:02:41.737,NaN,Nurse,Medication,999970898698810,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
3943957,Zonisamide Cap,Performed,835401600270,199570000200,2016-03-02 05:52:32.000,2016-02-04 11:47:24.923,NaN,Resident,Medication,999957778098810,NaN,NaN
3943958,Zonisamide Cap,Performed,835401600270,199570000200,2016-03-16 16:51:40.000,2016-02-04 11:47:24.923,NaN,Resident,Medication,999957778098810,NaN,NaN
3943959,Zonisamide Cap,Performed,835401600270,199570000200,2016-03-28 06:46:37.000,2016-02-04 11:47:24.923,NaN,Resident,Medication,999957778098810,NaN,NaN
3943960,Zonisamide Cap,Performed,1232743400270,2811200200,2018-09-23 17:27:37.000,2018-09-22 17:33:20.147,NaN,Resident,Medication,999788832998810,NaN,NaN


In [255]:
medication_opioid_performed = medication_opioid[medication_opioid['TaskStatusCode']=='Performed']

In [256]:
medication_opioid_performed

,name,TaskStatusCode,clientvisitguid,clientguid,PerformedFromDtm,createdwhen,ordersetname,enterrole,typecode,userguid,Generic,OrderName
63,(Omnicell Override No Task),Performed,1167028600270,227099700200,2015-06-02 10:18:00.000,2015-06-02 10:19:24.437,NaN,Nurse,Medication,999970898698810,NaN,NaN
64,(Omnicell Override No Task),Performed,1171775000270,229031900200,2015-06-12 07:16:00.000,2015-06-12 07:17:01.493,NaN,Nurse,Medication,999970898698810,NaN,NaN
65,(Omnicell Override No Task),Performed,1167028600270,227099700200,2015-06-02 07:25:00.000,2015-06-02 07:25:43.717,NaN,Nurse,Medication,999970898698810,NaN,NaN
66,(Omnicell Override No Task),Performed,1171775000270,229031900200,2015-06-12 07:34:00.000,2015-06-12 07:34:59.600,NaN,Nurse,Medication,999970898698810,NaN,NaN
67,(Omnicell Override No Task),Performed,1179389900270,6597000200,2015-07-28 15:48:00.000,2015-07-28 15:49:57.250,NaN,Nurse,Medication,999970898698810,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
3943957,Zonisamide Cap,Performed,835401600270,199570000200,2016-03-02 05:52:32.000,2016-02-04 11:47:24.923,NaN,Resident,Medication,999957778098810,NaN,NaN
3943958,Zonisamide Cap,Performed,835401600270,199570000200,2016-03-16 16:51:40.000,2016-02-04 11:47:24.923,NaN,Resident,Medication,999957778098810,NaN,NaN
3943959,Zonisamide Cap,Performed,835401600270,199570000200,2016-03-28 06:46:37.000,2016-02-04 11:47:24.923,NaN,Resident,Medication,999957778098810,NaN,NaN
3943960,Zonisamide Cap,Performed,1232743400270,2811200200,2018-09-23 17:27:37.000,2018-09-22 17:33:20.147,NaN,Resident,Medication,999788832998810,NaN,NaN


In [257]:
medication_opioid_performed_os = medication_opioid_performed.groupby(['clientvisitguid', 'Generic'])[['ordersetname']].apply(lambda x: x.notnull().sum())

In [258]:
medication_opioid_performed_os = medication_opioid_performed_os.reset_index()

In [259]:
medication_opioid_performed_os = medication_opioid_performed_os.pivot_table(index='clientvisitguid', columns='Generic', values='ordersetname')

In [260]:
medication_opioid_performed_os

Generic,Codeine,Diphenoxylate,Fentanyl,Hydrocodone,Hydromorphone,Meperidine,Methadone,Morphine,Oxycodone
clientvisitguid,,,,,,,,,
503942200270,NaN,NaN,0.0,NaN,1.0,0.0,NaN,0.0,0.0
529725300270,NaN,NaN,0.0,NaN,14.0,NaN,NaN,0.0,0.0
541079700270,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,0.0
544801700270,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,0.0
546823500270,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...
1476885700270,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
1476914500270,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1477569900270,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN


In [261]:
medication_opioid_performed_os = medication_opioid_performed_os.fillna(0)

In [262]:
medication_opioid_performed_os = medication_opioid_performed_os.reset_index()

In [263]:
medication_opioid_performed_os.columns

Index(['clientvisitguid', 'Codeine', 'Diphenoxylate', 'Fentanyl',
       'Hydrocodone', 'Hydromorphone', 'Meperidine', 'Methadone', 'Morphine',
       'Oxycodone'],
      dtype='object', name='Generic')

In [264]:
opioid_order_os = pd.merge(left=table_opioidonly_performed, right=medication_opioid_performed_os, 
                           on='clientvisitguid', how='left')

In [265]:
opioid_order_os = opioid_order_os.fillna(0)

In [266]:
opioid_order_os

,clientvisitguid,"Performed, Codeine","Performed, Diphenoxylate","Performed, Fentanyl","Performed, Hydrocodone","Performed, Hydromorphone","Performed, Meperidine","Performed, Methadone","Performed, Morphine","Performed, Oxycodone",Codeine,Diphenoxylate,Fentanyl,Hydrocodone,Hydromorphone,Meperidine,Methadone,Morphine,Oxycodone
0,1000120800270,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1000124200270,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1000155300270,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1000634700270,0,0,4,0,0,0,0,0,0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1001023900270,0,0,0,0,0,0,0,0,16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1646,999130900270,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1647,999139900270,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1648,999619900270,0,0,7,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1649,999912100270,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [267]:
opioid_order_os.columns = ['clientvisitguid', 'Codeine_p_order', 'Diphenoxylate_p_order', 'Fentanyl_p_order', 
                          'Hydrocodone_p_order', 'Hydromorphone_p_order', 'Meperidine_p_order', 'Methadone_p_order', 
                          'Morphine_p_order', 'Oxycodone_p_order', 'Codeine_p_OS', 'Diphenoxylate_p_OS', 'Fentanyl_p_OS', 
                          'Hydrocodone_p_OS', 'Hydromorphone_p_OS', 'Meperidine_p_OS', 'Methadone_p_OS', 'Morphine_p_OS', 
                          'Oxycodone_p_OS']

In [268]:
opioid_order_os

,clientvisitguid,Codeine_p_order,Diphenoxylate_p_order,Fentanyl_p_order,Hydrocodone_p_order,Hydromorphone_p_order,Meperidine_p_order,Methadone_p_order,Morphine_p_order,Oxycodone_p_order,Codeine_p_OS,Diphenoxylate_p_OS,Fentanyl_p_OS,Hydrocodone_p_OS,Hydromorphone_p_OS,Meperidine_p_OS,Methadone_p_OS,Morphine_p_OS,Oxycodone_p_OS
0,1000120800270,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1000124200270,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1000155300270,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1000634700270,0,0,4,0,0,0,0,0,0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1001023900270,0,0,0,0,0,0,0,0,16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1646,999130900270,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1647,999139900270,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1648,999619900270,0,0,7,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1649,999912100270,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [269]:
opioid_order_os.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1651 entries, 0 to 1650
Data columns (total 19 columns):
clientvisitguid          1651 non-null int64
Codeine_p_order          1651 non-null int64
Diphenoxylate_p_order    1651 non-null int64
Fentanyl_p_order         1651 non-null int64
Hydrocodone_p_order      1651 non-null int64
Hydromorphone_p_order    1651 non-null int64
Meperidine_p_order       1651 non-null int64
Methadone_p_order        1651 non-null int64
Morphine_p_order         1651 non-null int64
Oxycodone_p_order        1651 non-null int64
Codeine_p_OS             1651 non-null float64
Diphenoxylate_p_OS       1651 non-null float64
Fentanyl_p_OS            1651 non-null float64
Hydrocodone_p_OS         1651 non-null float64
Hydromorphone_p_OS       1651 non-null float64
Meperidine_p_OS          1651 non-null float64
Methadone_p_OS           1651 non-null float64
Morphine_p_OS            1651 non-null float64
Oxycodone_p_OS           1651 non-null float64
dtypes: float64(9), 

In [272]:
opioid_order_os.to_excel(path + 'opioid_order_orderset_1651.xlsx', index=None)

In [270]:
opioid_order_os['Codeine_p_OS/order'] = opioid_order_os['Codeine_p_OS']/opioid_order_os['Codeine_p_order']

In [271]:
opioid_order_os['Codeine_p_OS/order'].max()

0.0

In [107]:
opioid_order_os['Diphenoxylate_p_OS/order'] = opioid_order_os['Diphenoxylate_p_OS']/opioid_order_os['Diphenoxylate_p_order']

In [113]:
opioid_order_os['Diphenoxylate_p_OS/order']

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
        ..
1646   NaN
1647   NaN
1648   NaN
1649   NaN
1650   NaN
Name: Diphenoxylate_p_OS/order, Length: 1651, dtype: float64

In [109]:
opioid_order_os['Fentanyl_p_OS/order'] = opioid_order_os['Fentanyl_p_OS']/opioid_order_os['Fentanyl_p_order']

In [111]:
opioid_order_os['Fentanyl_p_OS/order'].describe()

count    310.000000
mean       0.342089
std        0.456840
min        0.000000
25%        0.000000
50%        0.000000
75%        1.000000
max        1.000000
Name: Fentanyl_p_OS/order, dtype: float64